In [6]:
import os
import numpy as np
from IPython.core.debugger import set_trace
import importlib
import sys
from default_config.masif_opts import masif_opts
from masif_modules.MaSIF_ligand import MaSIF_ligand
from masif_modules.MaSIF_ligand_Brian_tf1 import MaSIF_ligand2
from masif_modules.read_ligand_tfrecords_tf1 import _parse_function
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from default_config.masif_opts import masif_opts


custom_params = {}
custom_params['cache_dir'] = 'nn_models/sc05/cache/'
custom_params['model_dir'] = 'nn_models/sc05/all_feat/model_data/'
custom_params['desc_dir'] = 'output/sc05/all_feat/model_data/'
custom_params['gif_eval_out'] = 'nn_models/sc05/gif_eval/'
custom_params['min_sc_filt'] = 0.5
custom_params['max_sc_filt'] = 1.0
custom_params['pos_surf_accept_probability'] = 1.0

params = masif_opts["site"]
params["n_conv_layers"]=1
for key in custom_params:
    print("Setting {} to {} ".format(key, custom_params[key]))
    params[key] = custom_params[key]


params["max_distance"]=5


Setting cache_dir to nn_models/sc05/cache/ 
Setting model_dir to nn_models/sc05/all_feat/model_data/ 
Setting desc_dir to output/sc05/all_feat/model_data/ 
Setting gif_eval_out to nn_models/sc05/gif_eval/ 
Setting min_sc_filt to 0.5 
Setting max_sc_filt to 1.0 
Setting pos_surf_accept_probability to 1.0 


In [17]:
import glob
data = "/data/pompei/bw973/Oxygenases/PHD2/PHD2_50_O2IF/Bundle/Sim1"

ppi_pair_ids=glob.glob(data+"/data_preparation/04a-precomputation_12A/precomputation/Sim_1_traj_*")
ppi_pair_ids=[i.split('/')[-1] for i in ppi_pair_ids]
len(ppi_pair_ids)


6764

In [11]:
# ppi_pair_ids

In [18]:
batch_size=200
B=2000
params["n_conv_layers"]=1

ml = []
ml2 = {}

# ppi_pair_id="Sim_1_traj_42_frame_1307"
for i, ppi_pair_id in enumerate(ppi_pair_ids[0:B]):
    try:
        if i % 50 == 0:
            print(i)
        mydir=data+"/data_preparation/04a-precomputation_12A/precomputation/" + ppi_pair_id + "/"
        pdbid = ppi_pair_id.split(".")[0]
        pid=pdbid
        rho_wrt_center = np.load(mydir + pid + "_rho_wrt_center.npy")
        theta_wrt_center = np.load(mydir + pid + "_theta_wrt_center.npy")
        input_feat = np.load(mydir + pid + "_input_feat.npy")

        iface_labels = np.load(mydir + pid + "_iface_labels.npy")

        mask = np.load(mydir + pid + "_mask.npy")
        mask = np.expand_dims(mask, 2)
        indices = np.load(mydir + pid + "_list_indices.npy", allow_pickle=True).item()['protein']

        # indices is (n_verts x <30), it should be
        # indices = pad_indices(indices, mask.shape[1])
        tmp = np.zeros((len(iface_labels), 3))
        for i in range(len(iface_labels)):
            if iface_labels[i] == 1:
                tmp[i, 1] = 1
            elif iface_labels[i] == 2:
                tmp[i, 2] = 1
            else:
                tmp[i, 0] = 1
        iface_labels_dc = tmp
        super_pos = np.where(iface_labels == 2)[0]
        pos_labels = np.where(iface_labels == 1)[0]
        neg_labels = np.where(iface_labels == 0)[0]
        # print("PNS",len(pos_labels),len(neg_labels), len(super_pos))


        n = min(len(pos_labels), len(neg_labels))
        # print(ppi_pair_id,"len(pos_labels)", len(pos_labels), 'len(super_pos)', len(super_pos),'len(neg_labels)', len(neg_labels))
        n = min(n, batch_size // 2)
        n_neg = 500-n-len(super_pos)
        # this is 
        subset = np.concatenate([neg_labels[:n_neg], pos_labels[:n], super_pos])
        # print(len(subset))

        rho_wrt_center = rho_wrt_center[subset]
        theta_wrt_center = theta_wrt_center[subset]
        input_feat = input_feat[subset]
        mask = mask[subset]
        iface_labels_dc = iface_labels_dc[subset]
        indices = indices[subset]
        # neg_labels = range(0, (n*5))
        # pos_labels = range((n*5), (n*5)+n)
        # super_pos_labels = range(n+(n*5),n+(n*5)+len(super_pos))
        neg_labels = range(0, n_neg)
        pos_labels = range(n_neg, n_neg+n)
        super_pos_labels = range(n+(n_neg),n+(n_neg)+len(super_pos))
        # print("PNS labels",len(neg_labels),len(pos_labels), len(super_pos_labels))
        # print(ppi_pair_id,"len(pos_labels)", len(pos_labels[:n]), 'len(super_pos_labels)', len(super_pos_labels),'len(neg_labels)', len(neg_labels[:n_neg]), len(np.concatenate([neg_labels,pos_labels,super_pos_labels])))
        assert(len(subset) == 500)

        ml.append([input_feat,rho_wrt_center,theta_wrt_center,mask,iface_labels_dc])
        ml2[ppi_pair_id] = {
            'input_feat':input_feat,
            'rho_wrt_center':rho_wrt_center,
            'theta_wrt_center':theta_wrt_center,
            'mask':mask,
            'iface_labels_dc':iface_labels_dc
        }
    except Exception as e:
        print("Error", e, ppi_pair_id)

np.save(data+"/inputs_B%d.npy"%B, ml2)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950


In [15]:
np.save(data+"/inputs_B%d.npy"%B, ml2)

In [ ]:
# ml2 = np.load("inputs_B%d.npy"%B, allow_pickle=True).item()
# 

In [ ]:
np.save("inputs_B%d.npy"%B, ml2)

In [ ]:
6000/B*769/1000

In [ ]:
# import tensorflow as tf
# import tensorflow.keras as keras


# class MyModel(keras.Model):
#   def __init__(self, n_rotations=1):
#     super(MyModel, self).__init__()
#     self.n_rotations=n_rotations,
#     self.conv1 = MaSIF_site2(
#         params["max_distance"],
#         n_thetas=4,
#         n_rhos=3,
#         n_rotations=self.n_rotations,
#         idx_gpu="/gpu:1",
#         feat_mask=[1.0]*5,
#         n_conv_layers=params["n_conv_layers"],
#     )

#   def call(self, inputs):
#     logits = self.conv1(inputs)
#     return logits

In [ ]:
TRAIN_TEST_SPLIT = 0.7

class SimDataGenerator():
    """
    Data generator for the UTKFace dataset. This class should be used when training our Keras multi-output model.
    """
    def __init__(self, list_):
        self.list_ = list_
        
    def generate_split_indexes(self):
        p = np.random.permutation(len(self.list_))
        train_up_to = int(len(self.list_) * TRAIN_TEST_SPLIT)
        train_idx = p[:train_up_to]
        test_idx = p[train_up_to:]

        train_up_to = int(train_up_to * TRAIN_TEST_SPLIT)
        train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]
        
        # # converts alias to id
        # self.df['gender_id'] = self.df['gender'].map(lambda gender: dataset_dict['gender_alias'][gender])
        # self.df['race_id'] = self.df['race'].map(lambda race: dataset_dict['race_alias'][race])

        # self.max_age = self.df['age'].max()
        
        return train_idx, valid_idx, test_idx

    def generate_frames(self, frame_idx, is_training, batch_size=16):
        """
        Used to generate a batch with images when training/testing/validating our Keras model.
        """
        
        # arrays to store our batched data
        input_feats, rhos, thetas, masks, labels = [], [], [], [], []
        while True:
            for idx in frame_idx:
                sample = self.list_[idx]
                
                input_feat = sample[0]
                rho = sample[1]
                theta = sample[2]
                mask = sample[3]
                label = sample[4]
                
                input_feats.append(input_feat)
                rhos.append(rho)
                thetas.append(theta)
                masks.append(mask)
                labels.append(label)
                
                # yielding condition
                if len(input_feats) >= batch_size:
                    # print("True")
                    yield (np.array(input_feats), np.array(rhos), np.array(thetas), np.array(masks)), np.array(labels)
                    input_feats, rhos, thetas, masks, labels = [], [], [], [], []
                    
            if not is_training:
                break
                
data_generator = SimDataGenerator(ml)
train_idx, valid_idx, test_idx = data_generator.generate_split_indexes()

batch_size = 32
valid_batch_size = 32
train_gen = data_generator.generate_frames(train_idx, is_training=True, batch_size=batch_size)
valid_gen = data_generator.generate_frames(valid_idx, is_training=True, batch_size=valid_batch_size)
test_gen = data_generator.generate_frames(test_idx, is_training=False, batch_size=valid_batch_size)

In [ ]:
class MaSIF_site2(tf.keras.layers.Layer):

    def inference(
        self,
        input_feat, # learning_obj.input_feat
        rho_coords, # learning_obj.rho_coords
        theta_coords,
        mask,
        W_conv,
        b_conv,
        mu_rho,
        sigma_rho,
        mu_theta,
        sigma_theta,
        eps=1e-5,
        mean_gauss_activation=True,
    ):
        n_samples = rho_coords.shape[1]
        n_vertices = rho_coords.shape[2]
        # print("n_samples", rho_coords.shape)
        # print("vertices", n_vertices)

        all_conv_feat = []
        for k in range(self.n_rotations):
            # print("rotation", k)
            # print("\trotation", k+1)
            # rho_coords_ = tf.reshape(rho_coords, [-1, 1])  # batch_size*n_vertices
            rho_coords_ = tf.reshape(rho_coords, (-1,n_samples*n_vertices, 1))
            # print("Brian rho_coords,rho_coords_", rho_coords.shape, rho_coords_.shape)
            # thetas_coords_ = tf.reshape(theta_coords, [-1, 1])  # batch_size*n_vertices
            thetas_coords_ = tf.reshape(theta_coords, (-1,n_samples*n_vertices,1))
            # print("thetas_coords,thetas_coords_", theta_coords.shape, thetas_coords_.shape)
            thetas_coords_ += k * 2 * np.pi / self.n_rotations
            thetas_coords_ = tf.math.mod(thetas_coords_, 2 * np.pi)
            rho_coords_ = tf.exp(
                -tf.square(rho_coords_ - mu_rho) / (tf.square(sigma_rho) + eps)
            )
            thetas_coords_ = tf.exp(
                -tf.square(thetas_coords_ - mu_theta) / (tf.square(sigma_theta) + eps)
            )
            
            gauss_activations = tf.multiply(
                rho_coords_, thetas_coords_
            )  # batch_size*n_vertices, n_gauss
            # print("gauss_activations shape 1",gauss_activations.shape)
            # print("gauss_activations1", gauss_activations.shape, rho_coords_.shape, thetas_coords_.shape)
            gauss_activations = tf.reshape(
                # gauss_activations, [n_samples, n_vertices, -1]
                gauss_activations,[-1,n_samples, n_vertices ,tf.shape(gauss_activations)[-1]]
            )  # batch_size, n_vertices, n_gauss
            # print("gauss_activations2", gauss_activations.shape, n_samples, n_vertices,-1)
            # print("gauss_activations shape 2",gauss_activations.shape)
            # print("gauss_activations,mask", gauss_activations.shape, mask.shape)
            gauss_activations = tf.multiply(gauss_activations, mask)
            # print("this is actually working")
            
            if (
                mean_gauss_activation
            ):  # computes mean weights for the different gaussians
                gauss_activations /= (
                    tf.reduce_sum(gauss_activations, 1, keepdims=True) + eps
                )  # batch_size, n_vertices, n_gauss
            # print("1", gauss_activations.shape)
            gauss_activations = tf.expand_dims(
                # gauss_activations, 2
                gauss_activations, 3
            )  # batch_size, inputsize, n_vertices, 1, n_gauss,
            # print("2", gauss_activations.shape)
            input_feat_ = tf.expand_dims(
                # input_feat, 3
                input_feat, 4
            )  # batch_size, inputsize, n_vertices, n_feat, 1
            # print("3", input_feat_.shape)
            gauss_desc = tf.multiply(
                gauss_activations, input_feat_
            )  # batch_size, n_vertices, n_feat, n_gauss,
            # print("4", gauss_desc.shape)
            gauss_desc = tf.reduce_sum(gauss_desc, 2)  # batch_size, n_feat, n_gauss,
            # print("5", gauss_desc.shape)
            gauss_desc = tf.reshape(
                gauss_desc, [-1,n_samples, self.n_thetas * self.n_rhos]
            )  # batch_size, 80
            
            # print("6", gauss_desc.shape)
            conv_feat = tf.matmul(gauss_desc, W_conv) + b_conv  # batch_size, 80
            # print("7", conv_feat.shape)
            # rho_coords2 = tf.identity(conv_feat)
            all_conv_feat.append(conv_feat)

        all_conv_feat = tf.stack(all_conv_feat)
        conv_feat = tf.reduce_max(all_conv_feat, 0)
        conv_feat = tf.nn.relu(conv_feat)
        return conv_feat
        # return 1,2
        
    def __init__(
        self,
        max_rho,
        n_thetas=4,
        n_rhos=3,
        n_gamma=1.0,
        learning_rate=1e-3,
        n_rotations=1,
        idx_gpu="/device:GPU:0",
        feat_mask=[1.0, 1.0, 1.0, 1.0, 1.0],
        n_conv_layers=1,
    ):
        super().__init__()
        self.max_rho=max_rho
       
        self.n_rhos=n_rhos
        self.n_thetas=n_thetas
        self.sigma_rho_init = (
            max_rho / 8
        ) 
        self.sigma_theta_init = 1.0
        self.learning_rate=learning_rate
        self.n_rotations=n_rotations
        self.n_feat = int(sum(feat_mask))
        self.n_labels = 3

        initial_coords = self.compute_initial_coordinates()
        self.initial_coords = initial_coords
        mu_rho_initial = np.expand_dims(initial_coords[:, 0], 0).astype(
                    "float32"
                )
        mu_theta_initial = np.expand_dims(initial_coords[:, 1], 0).astype(
            "float32"
        )
        # print(mu_rho_initial[0:100])
        self.mu_rho = []
        self.mu_theta = []
        self.sigma_rho = []
        self.sigma_theta = []
        for i in range(self.n_feat):
            self.mu_rho.append(
                tf.Variable(mu_rho_initial, name="mu_rho_{}".format(i))
            )  # 1, n_gauss
            self.mu_theta.append(
                tf.Variable(mu_theta_initial, name="mu_theta_{}".format(i))
            )  # 1, n_gauss
            self.sigma_rho.append(
                tf.Variable(
                    np.ones_like(mu_rho_initial) * self.sigma_rho_init,
                    name="sigma_rho_{}".format(i),
                )
            )  # 1, n_gauss
            self.sigma_theta.append(
                tf.Variable(
                    (np.ones_like(mu_theta_initial) * self.sigma_theta_init),
                    name="sigma_theta_{}".format(i),
                )
            )  # 1, n_gauss
        
        
        self.b_conv = []
        for i in range(self.n_feat):
            self.b_conv.append(
                tf.Variable(
                    tf.zeros([self.n_thetas * self.n_rhos]),
                    name="b_conv_{}".format(i),
                )
            )

        self.W_conv = []
        
        for i in range(self.n_feat):
            initializer=tf.keras.initializers.GlorotUniform(seed=(i+1)*100)
            self.W_conv.append(
                    tf.Variable(initializer((self.n_thetas * self.n_rhos, 
                                                self.n_thetas * self.n_rhos)),
                                name="W_conv_{}".format(i)
                                )
            )
        self.mlp2= tf.keras.layers.Dense(
                    self.n_thetas * self.n_rhos,
                    activation=tf.nn.relu,
                )
        self.mlp3= tf.keras.layers.Dense(
                    self.n_feat,
                    activation=tf.nn.relu,
                )
        self.mlp4= tf.keras.layers.Dense(
                    self.n_labels, activation='softmax'
                )
              
        # self.b = self.add_weight(shape=(self.n_ligands*,), initializer="zeros", trainable=True)

    def call(self, inputs):
        # print("INPUTS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!", len(inputs))
        input_feat=inputs[0]
        print("\tinput_feat", input_feat.shape)
        rho_coords=inputs[1]
        # print("\trho_coords", rho_coords.shape)
        theta_coords=inputs[2]
        # print("\ttheta_coords", theta_coords.shape)
        mask=inputs[3]
        # print("\tmask", mask.shape)
        # self.keep_prob=inputs[4]
        # labels=inputs[5]
    
        self.global_desc_1 = []
        self.rho_coords2 = []
        for i in range(self.n_feat):
        # for i in range(1):
            # print('feature',['si', 'ddc', 'hbond', 'charge', 'hphob'][i])
            # my_input_feat = tf.keras.ops.expand_dims(input_feat[:, :, i], 2)
            my_input_feat = tf.keras.ops.expand_dims(input_feat[:, :, :, i], 3)
            # print("WHAT", my_input_feat.shape)
            conv_feat = self.inference(
                            my_input_feat,
                            rho_coords,
                            theta_coords,
                            mask,
                            self.W_conv[i],
                            self.b_conv[i],
                            self.mu_rho[i],
                            self.sigma_rho[i],
                            self.mu_theta[i],
                            self.sigma_theta[i],
                        )
            self.global_desc_1.append(conv_feat)

        self.global_desc_1 = tf.stack(self.global_desc_1, axis=2)  
        # print("8", self.global_desc_1.shape)

        self.global_desc_1 = tf.reshape(
                    self.global_desc_1, [-1,500, self.n_thetas * self.n_rhos * self.n_feat]
                )
        # print("9", self.global_desc_1.shape)

        self.global_desc_1=self.mlp2(self.global_desc_1)
        # print("10", self.global_desc_1.shape)

        self.global_desc_1=self.mlp3(self.global_desc_1)
        # print("11", self.global_desc_1.shape)
       
        self.logits=self.mlp4(self.global_desc_1)
        # print("shape logits", self.logits.shape)
       
        # return self.global_desc_2, self.logits, self.rho_coords2
        return self.logits


    def compute_initial_coordinates(self):
        range_rho = [0.0, self.max_rho]
        range_theta = [0, 2 * np.pi]

        grid_rho = np.linspace(range_rho[0], range_rho[1], num=self.n_rhos + 1)
        grid_rho = grid_rho[1:]
        grid_theta = np.linspace(range_theta[0], range_theta[1], num=self.n_thetas + 1)
        grid_theta = grid_theta[:-1]

        grid_rho_, grid_theta_ = np.meshgrid(grid_rho, grid_theta, sparse=False)
        grid_rho_ = (
            grid_rho_.T
        )  # the traspose here is needed to have the same behaviour as Matlab code
        grid_theta_ = (
            grid_theta_.T
        )  # the traspose here is needed to have the same behaviour as Matlab code
        grid_rho_ = grid_rho_.flatten()
        grid_theta_ = grid_theta_.flatten()

        coords = np.concatenate((grid_rho_[None, :], grid_theta_[None, :]), axis=0)
        coords = coords.T  # every row contains the coordinates of a grid intersection
        # print(coords.shape)
        return coords

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras


class MyModel(keras.Model):
  def __init__(self, n_rotations=1):
    super(MyModel, self).__init__()
    self.n_rotations=n_rotations
    print("self.n_rotations", self.n_rotations)
    self.conv1 = MaSIF_site2(
        params["max_distance"],
        n_thetas=4,
        n_rhos=3,
        n_rotations=self.n_rotations,
        idx_gpu="/gpu:1",
        feat_mask=[1.0]*5,
        n_conv_layers=params["n_conv_layers"],
    )

  def call(self, inputs):
    logits = self.conv1(inputs)
    return logits

# inputA=keras.layers.Input((500,50,5))
# inputB=keras.layers.Input((500,50))
# inputC=keras.layers.Input((500,50))
# inputD=keras.layers.Input((500,50,1))

# model = MyModel(n_rotations=8)
# model([inputA,inputB,inputC,inputD])

In [ ]:
len(train_idx)//batch_size

In [ ]:
# inputA=keras.layers.Input((500,50,5))
# inputB=keras.layers.Input((500,50))
# inputC=keras.layers.Input((500,50))
# inputD=keras.layers.Input((500,50,1))

from keras.callbacks import ModelCheckpoint

model = MyModel(n_rotations=4)
# model([inputA,inputB,inputC,inputD])
callbacks = [
    ModelCheckpoint("./model_checkpoint.keras", monitor='val_loss')
]
model.compile(optimizer=keras.optimizers.Adam(learning_rate= 1e-3), 
    loss='categorical_crossentropy', 
    metrics = ['accuracy'])

model.fit(train_gen, steps_per_epoch=len(train_idx)//batch_size, epochs=5, callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=len(valid_idx)//valid_batch_size)

In [ ]:
# class MaSIF_site2(tf.keras.layers.Layer):

#     def inference(
#         self,
#         input_feat, # learning_obj.input_feat
#         rho_coords, # learning_obj.rho_coords
#         theta_coords,
#         mask,
#         W_conv,
#         b_conv,
#         mu_rho,
#         sigma_rho,
#         mu_theta,
#         sigma_theta,
#         eps=1e-5,
#         mean_gauss_activation=True,
#     ):
#         n_samples = tf.shape(rho_coords)[0]
#         n_vertices = tf.shape(rho_coords)[1]
#         print("n_samples",tf.shape(rho_coords))
#         print("vertices",n_vertices)

#         all_conv_feat = []
#         for k in range(self.n_rotations):
#             # print("\trotation", k+1)
#             rho_coords_ = tf.reshape(rho_coords, [-1, 1])  # batch_size*n_vertices
#             # rho_coords_ = tf.reshape(rho_coords, (-1,500*50, 1))
#             print("Brian rho_coords,rho_coords_", rho_coords.shape, rho_coords_.shape)
#             thetas_coords_ = tf.reshape(theta_coords, [-1, 1])  # batch_size*n_vertices
#             # thetas_coords_ = tf.reshape(theta_coords, (-1,500*50,1))
#             print("thetas_coords,thetas_coords_", theta_coords.shape, thetas_coords_.shape)
#             thetas_coords_ += k * 2 * np.pi / self.n_rotations
#             thetas_coords_ = tf.math.mod(thetas_coords_, 2 * np.pi)
#             rho_coords_ = tf.exp(
#                 -tf.square(rho_coords_ - mu_rho) / (tf.square(sigma_rho) + eps)
#             )
#             thetas_coords_ = tf.exp(
#                 -tf.square(thetas_coords_ - mu_theta) / (tf.square(sigma_theta) + eps)
#             )
            
#             gauss_activations = tf.multiply(
#                 rho_coords_, thetas_coords_
#             )  # batch_size*n_vertices, n_gauss
#             # print("gauss_activations shape 1",gauss_activations.shape)
#             print("gauss_activations1", gauss_activations.shape, rho_coords_.shape, thetas_coords_.shape)
#             gauss_activations = tf.reshape(
#                 gauss_activations, [n_samples, n_vertices, -1]
#                 # gauss_activations,[-1,500, 50,tf.shape(gauss_activations)[-1]]
#             )  # batch_size, n_vertices, n_gauss
#             # print("gauss_activations2", gauss_activations.shape, n_samples, n_vertices,-1)
#             # print("gauss_activations shape 2",gauss_activations.shape)
#             print("gauss_activations,mask", gauss_activations.shape, mask.shape)
#             gauss_activations = tf.multiply(gauss_activations, mask)
#             print("this is actually working")
            
#             if (
#                 mean_gauss_activation
#             ):  # computes mean weights for the different gaussians
#                 gauss_activations /= (
#                     tf.reduce_sum(gauss_activations, 1, keepdims=True) + eps
#                 )  # batch_size, n_vertices, n_gauss
#             print("1", gauss_activations.shape)
#             gauss_activations = tf.expand_dims(
#                 gauss_activations, 2
#                 # gauss_activations, 3
#             )  # batch_size, inputsize, n_vertices, 1, n_gauss,
#             print("2", gauss_activations.shape)
#             input_feat_ = tf.expand_dims(
#                 input_feat, 3
#                 # input_feat, 4
#             )  # batch_size, inputsize, n_vertices, n_feat, 1
#             print("3", input_feat_.shape)
#             gauss_desc = tf.multiply(
#                 gauss_activations, input_feat_
#             )  # batch_size, n_vertices, n_feat, n_gauss,
#             print("4", gauss_desc.shape)
#             gauss_desc = tf.reduce_sum(gauss_desc, 1)  # batch_size, n_feat, n_gauss,
#             print("5", gauss_desc.shape)
#             gauss_desc = tf.reshape(
#                 gauss_desc, [n_samples, self.n_thetas * self.n_rhos]
#             )  # batch_size, 80
            
#             print("6", gauss_desc.shape)
#             conv_feat = tf.matmul(gauss_desc, W_conv) + b_conv  # batch_size, 80
#             print("7", conv_feat.shape)
#             rho_coords2 = tf.identity(conv_feat)
#             all_conv_feat.append(conv_feat)

#         all_conv_feat = tf.stack(all_conv_feat)
#         conv_feat = tf.reduce_max(all_conv_feat, 0)
#         conv_feat = tf.nn.relu(conv_feat)
#         return conv_feat, rho_coords2
#         # return 1,2
        
#     def __init__(
#         self,
#         max_rho,
#         n_thetas=4,
#         n_rhos=3,
#         n_gamma=1.0,
#         learning_rate=1e-3,
#         n_rotations=1,
#         idx_gpu="/device:GPU:0",
#         feat_mask=[1.0, 1.0, 1.0, 1.0, 1.0],
#         n_conv_layers=1,
#     ):
#         super().__init__()
#         self.max_rho=max_rho
       
#         self.n_rhos=n_rhos
#         self.n_thetas=n_thetas
#         self.sigma_rho_init = (
#             max_rho / 8
#         ) 
#         self.sigma_theta_init = 1.0
#         self.learning_rate=learning_rate
#         self.n_rotations=n_rotations
#         self.n_feat = int(sum(feat_mask))
#         self.n_labels = 3

#         initial_coords = self.compute_initial_coordinates()
#         self.initial_coords = initial_coords
#         mu_rho_initial = np.expand_dims(initial_coords[:, 0], 0).astype(
#                     "float32"
#                 )
#         mu_theta_initial = np.expand_dims(initial_coords[:, 1], 0).astype(
#             "float32"
#         )
#         # print(mu_rho_initial[0:100])
#         self.mu_rho = []
#         self.mu_theta = []
#         self.sigma_rho = []
#         self.sigma_theta = []
#         for i in range(self.n_feat):
#             self.mu_rho.append(
#                 tf.Variable(mu_rho_initial, name="mu_rho_{}".format(i))
#             )  # 1, n_gauss
#             self.mu_theta.append(
#                 tf.Variable(mu_theta_initial, name="mu_theta_{}".format(i))
#             )  # 1, n_gauss
#             self.sigma_rho.append(
#                 tf.Variable(
#                     np.ones_like(mu_rho_initial) * self.sigma_rho_init,
#                     name="sigma_rho_{}".format(i),
#                 )
#             )  # 1, n_gauss
#             self.sigma_theta.append(
#                 tf.Variable(
#                     (np.ones_like(mu_theta_initial) * self.sigma_theta_init),
#                     name="sigma_theta_{}".format(i),
#                 )
#             )  # 1, n_gauss
        
        
#         self.b_conv = []
#         for i in range(self.n_feat):
#             self.b_conv.append(
#                 tf.Variable(
#                     tf.zeros([self.n_thetas * self.n_rhos]),
#                     name="b_conv_{}".format(i),
#                 )
#             )

#         self.W_conv = []
#         initializer=tf.keras.initializers.glorot_normal(seed=0)
#         for i in range(self.n_feat):
#             self.W_conv.append(
#                     tf.Variable(glorot_uniform((self.n_thetas * self.n_rhos, 
#                                                 self.n_thetas * self.n_rhos),
#                                             seed=(i+1)*100),
#                                 name="W_conv_{}".format(i)
#                                 )
#             )
#         self.mlp2= tf.keras.layers.Dense(
#                     self.n_thetas * self.n_rhos,
#                     activation=tf.nn.relu,
#                 )
#         self.mlp3= tf.keras.layers.Dense(
#                     self.n_feat,
#                     activation=tf.nn.relu,
#                 )
#         self.mlp4= tf.keras.layers.Dense(
#                     self.n_labels, activation='softmax'
#                 )
              
#         # self.b = self.add_weight(shape=(self.n_ligands*,), initializer="zeros", trainable=True)

#     def call(self, inputs):
#         print("INPUTS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!", len(inputs))
#         input_feat=inputs[0]
#         print("\tinput_feat", input_feat.shape)
#         rho_coords=inputs[1]
#         print("\trho_coords", rho_coords.shape)
#         theta_coords=inputs[2]
#         print("\ttheta_coords", theta_coords.shape)
#         mask=inputs[3]
#         print("\tmask", mask.shape)
#         # self.keep_prob=inputs[4]
#         # labels=inputs[5]
    
#         self.global_desc_1 = []
#         self.rho_coords2 = []
#         for i in range(self.n_feat):
#         # for i in range(1):
#             # print('feature',['si', 'ddc', 'hbond', 'charge', 'hphob'][i])
#             my_input_feat = tf.keras.ops.expand_dims(input_feat[:, :, i], 2)
#             # my_input_feat = tf.keras.ops.expand_dims(input_feat[:, :, :, i], 3)
#             print("WHAT", my_input_feat.shape)
#             conv_feat, rho_coords2, = self.inference(
#                             my_input_feat,
#                             rho_coords,
#                             theta_coords,
#                             mask,
#                             self.W_conv[i],
#                             self.b_conv[i],
#                             self.mu_rho[i],
#                             self.sigma_rho[i],
#                             self.mu_theta[i],
#                             self.sigma_theta[i],
#                         )
#             self.global_desc_1.append(conv_feat)
            

#         self.global_desc_1 = tf.stack(self.global_desc_1, axis=1)
#         print("8", self.global_desc_1.shape)
#         self.global_desc_1 = tf.reshape(
#                     self.global_desc_1, [-1, self.n_thetas * self.n_rhos * self.n_feat]
#                 )
#         print("9", self.global_desc_1.shape)
#         self.global_desc_1=self.mlp2(self.global_desc_1)
#         # self.global_desc_1 = tf.keras.layers.Dense(
#         #             self.n_thetas * self.n_rhos,
#         #             activation=tf.nn.relu,
#         #         )(self.global_desc_1)
#         print("10", self.global_desc_1.shape)
#         self.global_desc_1=self.mlp3(self.global_desc_1)
#         print("11", self.global_desc_1.shape)
#         # self.global_desc_1 = tf.keras.layers.Dense(
#         #             self.n_feat, activation=tf.nn.relu
#         #         )(self.global_desc_1)
#         self.logits=self.mlp4(self.global_desc_1)
#         print("shape logits", self.logits.shape)
#         # self.logits = tf.keras.layers.Dense(
#         #             self.n_labels, activation='softmax'
#         # )(self.global_desc_1)
        
#         # return self.global_desc_2, self.logits, self.rho_coords2
#         return self.logits

    
#     def compute_initial_coordinates(self):
#         range_rho = [0.0, self.max_rho]
#         range_theta = [0, 2 * np.pi]

#         grid_rho = np.linspace(range_rho[0], range_rho[1], num=self.n_rhos + 1)
#         grid_rho = grid_rho[1:]
#         grid_theta = np.linspace(range_theta[0], range_theta[1], num=self.n_thetas + 1)
#         grid_theta = grid_theta[:-1]

#         grid_rho_, grid_theta_ = np.meshgrid(grid_rho, grid_theta, sparse=False)
#         grid_rho_ = (
#             grid_rho_.T
#         )  # the traspose here is needed to have the same behaviour as Matlab code
#         grid_theta_ = (
#             grid_theta_.T
#         )  # the traspose here is needed to have the same behaviour as Matlab code
#         grid_rho_ = grid_rho_.flatten()
#         grid_theta_ = grid_theta_.flatten()

#         coords = np.concatenate((grid_rho_[None, :], grid_theta_[None, :]), axis=0)
#         coords = coords.T  # every row contains the coordinates of a grid intersection
#         print(coords.shape)
#         return coords

# model = MyModel()
# logits = model([input_feat,rho,theta,mask])

In [ ]:
test_gen = data_generator.generate_frames(test_idx, is_training=False, batch_size=valid_batch_size)

model.evaluate(test_gen, steps=len(test_idx)//valid_batch_size, 
               callbacks=callbacks)


In [ ]:
class MaSIF_site2(tf.keras.layers.Layer):

    def inference(
        self,
        input_feat, # learning_obj.input_feat
        rho_coords, # learning_obj.rho_coords
        theta_coords,
        mask,
        W_conv,
        b_conv,
        mu_rho,
        sigma_rho,
        mu_theta,
        sigma_theta,
        eps=1e-5,
        mean_gauss_activation=True,
    ):
        n_samples = rho_coords.shape[1]
        n_vertices = rho_coords.shape[2]
        # print("n_samples", rho_coords.shape)
        # print("vertices", n_vertices)

        all_conv_feat = []
        for k in range(self.n_rotations):
            print("rotation", k)
            # print("\trotation", k+1)
            # rho_coords_ = tf.reshape(rho_coords, [-1, 1])  # batch_size*n_vertices
            rho_coords_ = tf.reshape(rho_coords, (-1,n_samples*n_vertices, 1))
            # print("Brian rho_coords,rho_coords_", rho_coords.shape, rho_coords_.shape)
            # thetas_coords_ = tf.reshape(theta_coords, [-1, 1])  # batch_size*n_vertices
            thetas_coords_ = tf.reshape(theta_coords, (-1,n_samples*n_vertices,1))
            # print("thetas_coords,thetas_coords_", theta_coords.shape, thetas_coords_.shape)
            thetas_coords_ += k * 2 * np.pi / self.n_rotations
            thetas_coords_ = tf.math.mod(thetas_coords_, 2 * np.pi)
            rho_coords_ = tf.exp(
                -tf.square(rho_coords_ - mu_rho) / (tf.square(sigma_rho) + eps)
            )
            thetas_coords_ = tf.exp(
                -tf.square(thetas_coords_ - mu_theta) / (tf.square(sigma_theta) + eps)
            )
            
            gauss_activations = tf.multiply(
                rho_coords_, thetas_coords_
            )  # batch_size*n_vertices, n_gauss
            # print("gauss_activations shape 1",gauss_activations.shape)
            # print("gauss_activations1", gauss_activations.shape, rho_coords_.shape, thetas_coords_.shape)
            gauss_activations = tf.reshape(
                # gauss_activations, [n_samples, n_vertices, -1]
                gauss_activations,[-1,n_samples, n_vertices ,tf.shape(gauss_activations)[-1]]
            )  # batch_size, n_vertices, n_gauss
            # print("gauss_activations2", gauss_activations.shape, n_samples, n_vertices,-1)
            # print("gauss_activations shape 2",gauss_activations.shape)
            # print("gauss_activations,mask", gauss_activations.shape, mask.shape)
            gauss_activations = tf.multiply(gauss_activations, mask)
            # print("this is actually working")
            
            if (
                mean_gauss_activation
            ):  # computes mean weights for the different gaussians
                gauss_activations /= (
                    tf.reduce_sum(gauss_activations, 1, keepdims=True) + eps
                )  # batch_size, n_vertices, n_gauss
            # print("1", gauss_activations.shape)
            gauss_activations = tf.expand_dims(
                # gauss_activations, 2
                gauss_activations, 3
            )  # batch_size, inputsize, n_vertices, 1, n_gauss,
            # print("2", gauss_activations.shape)
            input_feat_ = tf.expand_dims(
                # input_feat, 3
                input_feat, 4
            )  # batch_size, inputsize, n_vertices, n_feat, 1
            # print("3", input_feat_.shape)
            gauss_desc = tf.multiply(
                gauss_activations, input_feat_
            )  # batch_size, n_vertices, n_feat, n_gauss,
            # print("4", gauss_desc.shape)
            gauss_desc = tf.reduce_sum(gauss_desc, 2)  # batch_size, n_feat, n_gauss,
            # print("5", gauss_desc.shape)
            gauss_desc = tf.reshape(
                gauss_desc, [-1,n_samples, self.n_thetas * self.n_rhos]
            )  # batch_size, 80
            
            # print("6", gauss_desc.shape)
            conv_feat = tf.matmul(gauss_desc, W_conv) + b_conv  # batch_size, 80
            print("7", conv_feat.shape)
            # rho_coords2 = tf.identity(conv_feat)
            all_conv_feat.append(conv_feat)

        all_conv_feat = tf.stack(all_conv_feat)
        conv_feat = tf.reduce_max(all_conv_feat, 0)
        conv_feat = tf.nn.relu(conv_feat)
        return conv_feat
        # return 1,2
        
    def __init__(
        self,
        max_rho,
        n_thetas=4,
        n_rhos=3,
        n_gamma=1.0,
        learning_rate=1e-3,
        n_rotations=1,
        idx_gpu="/device:GPU:0",
        feat_mask=[1.0, 1.0, 1.0, 1.0, 1.0],
        n_conv_layers=1,
    ):
        super().__init__()
        self.max_rho=max_rho
       
        self.n_rhos=n_rhos
        self.n_thetas=n_thetas
        self.sigma_rho_init = (
            max_rho / 8
        ) 
        self.sigma_theta_init = 1.0
        self.learning_rate=learning_rate
        self.n_rotations=n_rotations
        self.n_feat = int(sum(feat_mask))
        self.n_labels = 3

        initial_coords = self.compute_initial_coordinates()
        self.initial_coords = initial_coords
        mu_rho_initial = np.expand_dims(initial_coords[:, 0], 0).astype(
                    "float32"
                )
        mu_theta_initial = np.expand_dims(initial_coords[:, 1], 0).astype(
            "float32"
        )
        # print(mu_rho_initial[0:100])
        self.mu_rho = []
        self.mu_theta = []
        self.sigma_rho = []
        self.sigma_theta = []
        for i in range(self.n_feat):
            self.mu_rho.append(
                tf.Variable(mu_rho_initial, name="mu_rho_{}".format(i))
            )  # 1, n_gauss
            self.mu_theta.append(
                tf.Variable(mu_theta_initial, name="mu_theta_{}".format(i))
            )  # 1, n_gauss
            self.sigma_rho.append(
                tf.Variable(
                    np.ones_like(mu_rho_initial) * self.sigma_rho_init,
                    name="sigma_rho_{}".format(i),
                )
            )  # 1, n_gauss
            self.sigma_theta.append(
                tf.Variable(
                    (np.ones_like(mu_theta_initial) * self.sigma_theta_init),
                    name="sigma_theta_{}".format(i),
                )
            )  # 1, n_gauss
        
        
        self.b_conv = []
        for i in range(self.n_feat):
            self.b_conv.append(
                tf.Variable(
                    tf.zeros([self.n_thetas * self.n_rhos]),
                    name="b_conv_{}".format(i),
                )
            )

        self.W_conv = []
        
        for i in range(self.n_feat):
            initializer=tf.keras.initializers.GlorotUniform(seed=(i+1)*100)
            self.W_conv.append(
                    tf.Variable(initializer((self.n_thetas * self.n_rhos, 
                                                self.n_thetas * self.n_rhos)),
                                name="W_conv_{}".format(i)
                                )
            )
        self.mlp2= tf.keras.layers.Dense(
                    self.n_thetas * self.n_rhos,
                    activation=tf.nn.relu,
                )
        self.mlp3= tf.keras.layers.Dense(
                    self.n_feat,
                    activation=tf.nn.relu,
                )
        self.mlp4= tf.keras.layers.Dense(
                    self.n_labels, activation='softmax'
                )
              
        # self.b = self.add_weight(shape=(self.n_ligands*,), initializer="zeros", trainable=True)

    def call(self, inputs):
        # print("INPUTS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!", len(inputs))
        input_feat=inputs[0]
        print("\tinput_feat", input_feat.shape)
        rho_coords=inputs[1]
        # print("\trho_coords", rho_coords.shape)
        theta_coords=inputs[2]
        # print("\ttheta_coords", theta_coords.shape)
        mask=inputs[3]
        # print("\tmask", mask.shape)
        # self.keep_prob=inputs[4]
        # labels=inputs[5]
    
        self.global_desc_1 = []
        self.rho_coords2 = []
        for i in range(self.n_feat):
        # for i in range(1):
            print('feature',['si', 'ddc', 'hbond', 'charge', 'hphob'][i])
            # my_input_feat = tf.keras.ops.expand_dims(input_feat[:, :, i], 2)
            my_input_feat = tf.keras.ops.expand_dims(input_feat[:, :, :, i], 3)
            # print("WHAT", my_input_feat.shape)
            conv_feat = self.inference(
                            my_input_feat,
                            rho_coords,
                            theta_coords,
                            mask,
                            self.W_conv[i],
                            self.b_conv[i],
                            self.mu_rho[i],
                            self.sigma_rho[i],
                            self.mu_theta[i],
                            self.sigma_theta[i],
                        )
            self.global_desc_1.append(conv_feat)

        self.global_desc_1 = tf.stack(self.global_desc_1, axis=2)  
        # print("8", self.global_desc_1.shape)

        self.global_desc_1 = tf.reshape(
                    self.global_desc_1, [-1,500, self.n_thetas * self.n_rhos * self.n_feat]
                )
        # print("9", self.global_desc_1.shape)

        self.global_desc_1=self.mlp2(self.global_desc_1)
        # print("10", self.global_desc_1.shape)

        self.global_desc_1=self.mlp3(self.global_desc_1)
        # print("11", self.global_desc_1.shape)
       
        self.logits=self.mlp4(self.global_desc_1)
        print("shape logits", self.logits.shape)
       
        # return self.global_desc_2, self.logits, self.rho_coords2
        return self.logits


    def compute_initial_coordinates(self):
        range_rho = [0.0, self.max_rho]
        range_theta = [0, 2 * np.pi]

        grid_rho = np.linspace(range_rho[0], range_rho[1], num=self.n_rhos + 1)
        grid_rho = grid_rho[1:]
        grid_theta = np.linspace(range_theta[0], range_theta[1], num=self.n_thetas + 1)
        grid_theta = grid_theta[:-1]

        grid_rho_, grid_theta_ = np.meshgrid(grid_rho, grid_theta, sparse=False)
        grid_rho_ = (
            grid_rho_.T
        )  # the traspose here is needed to have the same behaviour as Matlab code
        grid_theta_ = (
            grid_theta_.T
        )  # the traspose here is needed to have the same behaviour as Matlab code
        grid_rho_ = grid_rho_.flatten()
        grid_theta_ = grid_theta_.flatten()

        coords = np.concatenate((grid_rho_[None, :], grid_theta_[None, :]), axis=0)
        coords = coords.T  # every row contains the coordinates of a grid intersection
        # print(coords.shape)
        return coords



In [ ]:
import tensorflow as tf
import tensorflow.keras as keras


class MyModel(keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = MaSIF_site2(
        params["max_distance"],
        n_thetas=4,
        n_rhos=3,
        n_rotations=1,
        idx_gpu="/gpu:1",
        feat_mask=[1.0]*5,
        n_conv_layers=params["n_conv_layers"],
    )

  def call(self, inputs):
    logits = self.conv1(inputs)
    return logits

In [ ]:
def glorot_uniform(shape, seed=None):
    if seed is not None:
        np.random.seed(seed)
    limit = np.sqrt(6 / (shape[0] + shape[1]))
    return np.random.uniform(-limit, limit, size=shape).astype(np.float32)

In [ ]:
class MaSIF_ligand2(tf.keras.layers.Layer):
    def __init__(
        self,
        max_rho,
        n_ligands,
        n_thetas=16,
        n_rhos=5,
        n_gamma=1.0,
        learning_rate=1e-4,
        n_rotations=16,
        idx_gpu="/gpu:0",
        feat_mask=[1.0, 1.0, 1.0, 1.0],
        costfun="dprime"
    ):
        super().__init__()
        self.max_rho=max_rho
        self.n_ligands=n_ligands
        self.n_rhos=n_rhos
        self.n_thetas=n_thetas
        self.sigma_rho_init = (
            max_rho / 8
        ) 
        self.sigma_theta_init = 1.0
        self.learning_rate=learning_rate
        self.n_rotations=n_rotations
        self.n_feat = int(sum(feat_mask))

        initial_coords = self.compute_initial_coordinates()
        mu_rho_initial = np.expand_dims(initial_coords[:, 0], 0).astype(
                    "float32"
                )
        mu_theta_initial = np.expand_dims(initial_coords[:, 1], 0).astype(
            "float32"
        )
        print(mu_rho_initial[0:100])
        self.mu_rho = []
        self.mu_theta = []
        self.sigma_rho = []
        self.sigma_theta = []
        for i in range(self.n_feat):
            self.mu_rho.append(
                tf.Variable(mu_rho_initial, name="mu_rho_{}".format(i))
            )  # 1, n_gauss
            self.mu_theta.append(
                tf.Variable(mu_theta_initial, name="mu_theta_{}".format(i))
            )  # 1, n_gauss
            self.sigma_rho.append(
                tf.Variable(
                    np.ones_like(mu_rho_initial) * self.sigma_rho_init,
                    name="sigma_rho_{}".format(i),
                )
            )  # 1, n_gauss
            self.sigma_theta.append(
                tf.Variable(
                    (np.ones_like(mu_theta_initial) * self.sigma_theta_init),
                    name="sigma_theta_{}".format(i),
                )
            )  # 1, n_gauss
        
        
        self.b_conv = []
        for i in range(self.n_feat):
            self.b_conv.append(
                tf.Variable(
                    tf.zeros([self.n_thetas * self.n_rhos]),
                    name="b_conv_{}".format(i),
                )
            )

        self.W_conv = []
        initializer=tf.keras.initializers.glorot_normal(seed=0)
        for i in range(self.n_feat):
            self.W_conv.append(
                    tf.Variable(glorot_uniform((80, 80), seed=(i+1)*100),
                                name="W_conv_{}".format(i)
                                )
            )
        # self.b = self.add_weight(shape=(self.n_ligands*,), initializer="zeros", trainable=True)

    def call(self, inputs):
        input_feat=inputs[0]
        rho_coords=inputs[1]
        theta_coords=inputs[2]
        mask=inputs[3]
        labels=inputs[4]
        self.keep_prob=inputs[5]
    
        self.global_desc_1 = []
        self.rho_coords2 = []
        for i in range(self.n_feat):
            print('feature',['si', 'ddc', 'hbond', 'charge', 'hphob'][i])
            my_input_feat = tf.keras.ops.expand_dims(input_feat[:, :, i], 2)
            conv_feat, rho_coords2, = self.inference(
                            my_input_feat,
                            rho_coords,
                            theta_coords,
                            mask,
                            self.W_conv[i],
                            self.b_conv[i],
                            self.mu_rho[i],
                            self.sigma_rho[i],
                            self.mu_theta[i],
                            self.sigma_theta[i],
                            
                        )
            self.global_desc_1.append(conv_feat)
            self.rho_coords2.append(rho_coords2)

        self.global_desc_1 = tf.stack(self.global_desc_1, axis=1)
        self.global_desc_1 = tf.reshape(
                    self.global_desc_1, [-1, self.n_thetas * self.n_rhos * self.n_feat]
                )
        
        self.global_desc_1 = tf.keras.layers.Dense(
                    self.n_thetas * self.n_rhos,
                    activation=tf.nn.relu,
                )(self.global_desc_1)

        self.global_desc_1 = tf.matmul(
                    tf.transpose(self.global_desc_1), self.global_desc_1
                ) / tf.cast(tf.shape(self.global_desc_1)[0], tf.float32)
        self.global_desc_2 = self.global_desc_1
        # self.global_desc_2 = self.global_desc_1
        # self.global_desc_2 = self.global_desc_1
        self.global_desc_1 = tf.reshape(self.global_desc_1, [1, -1])
        self.global_desc_1 = tf.nn.dropout(self.global_desc_1, self.keep_prob)
        self.global_desc_1 = tf.keras.layers.Dense(
            64, activation=tf.nn.relu
        )(self.global_desc_1)
        
        self.logits = tf.keras.layers.Dense(
                    self.n_ligands, activation=tf.identity
        )(self.global_desc_1)
        
        return self.global_desc_2, self.logits, self.rho_coords2

    def inference(
        self,
        input_feat, # learning_obj.input_feat
        rho_coords, # learning_obj.rho_coords
        theta_coords,
        mask,
        W_conv,
        b_conv,
        mu_rho,
        sigma_rho,
        mu_theta,
        sigma_theta,
        eps=1e-5,
        mean_gauss_activation=True,
    ):
        n_samples = tf.shape(rho_coords)[0]
        n_vertices = tf.shape(rho_coords)[1]
      

        all_conv_feat = []
        for k in range(self.n_rotations):
            print("\trotation", k+1)
            rho_coords_ = tf.reshape(rho_coords, [-1, 1])  # batch_size*n_vertices
            thetas_coords_ = tf.reshape(theta_coords, [-1, 1])  # batch_size*n_vertices

            thetas_coords_ += k * 2 * np.pi / self.n_rotations
            thetas_coords_ = tf.math.mod(thetas_coords_, 2 * np.pi)
            rho_coords_ = tf.exp(
                -tf.square(rho_coords_ - mu_rho) / (tf.square(sigma_rho) + eps)
            )
            thetas_coords_ = tf.exp(
                -tf.square(thetas_coords_ - mu_theta) / (tf.square(sigma_theta) + eps)
            )
            
            gauss_activations = tf.multiply(
                rho_coords_, thetas_coords_
            )  # batch_size*n_vertices, n_gauss
            # print("gauss_activations shape 1",gauss_activations.shape)
            
            gauss_activations = tf.reshape(
                gauss_activations, [n_samples, n_vertices, -1]
            )  # batch_size, n_vertices, n_gauss
            # print("gauss_activations shape 2",gauss_activations.shape)
            gauss_activations = tf.multiply(gauss_activations, mask)
            
            if (
                mean_gauss_activation
            ):  # computes mean weights for the different gaussians
                gauss_activations /= (
                    tf.reduce_sum(gauss_activations, 1, keepdims=True) + eps
                )  # batch_size, n_vertices, n_gauss
            
            gauss_activations = tf.expand_dims(
                gauss_activations, 2
            )  # batch_size, n_vertices, 1, n_gauss,
            input_feat_ = tf.expand_dims(
                input_feat, 3
            )  # batch_size, n_vertices, n_feat, 1

            gauss_desc = tf.multiply(
                gauss_activations, input_feat_
            )  # batch_size, n_vertices, n_feat, n_gauss,
            
            gauss_desc = tf.reduce_sum(gauss_desc, 1)  # batch_size, n_feat, n_gauss,
            gauss_desc = tf.reshape(
                gauss_desc, [n_samples, self.n_thetas * self.n_rhos]
            )  # batch_size, 80
            
            
            conv_feat = tf.matmul(gauss_desc, W_conv) + b_conv  # batch_size, 80
            rho_coords2 = tf.identity(conv_feat)
            all_conv_feat.append(conv_feat)

        all_conv_feat = tf.stack(all_conv_feat)
        conv_feat = tf.reduce_max(all_conv_feat, 0)
        conv_feat = tf.nn.relu(conv_feat)
        return conv_feat, rho_coords2


    def compute_initial_coordinates(self):
        range_rho = [0.0, self.max_rho]
        range_theta = [0, 2 * np.pi]

        grid_rho = np.linspace(range_rho[0], range_rho[1], num=self.n_rhos + 1)
        grid_rho = grid_rho[1:]
        grid_theta = np.linspace(range_theta[0], range_theta[1], num=self.n_thetas + 1)
        grid_theta = grid_theta[:-1]

        grid_rho_, grid_theta_ = np.meshgrid(grid_rho, grid_theta, sparse=False)
        grid_rho_ = (
            grid_rho_.T
        )  # the traspose here is needed to have the same behaviour as Matlab code
        grid_theta_ = (
            grid_theta_.T
        )  # the traspose here is needed to have the same behaviour as Matlab code
        grid_rho_ = grid_rho_.flatten()
        grid_theta_ = grid_theta_.flatten()

        coords = np.concatenate((grid_rho_[None, :], grid_theta_[None, :]), axis=0)
        coords = coords.T  # every row contains the coordinates of a grid intersection
        print(coords.shape)
        return coords

In [ ]:
tf.keras.utils.set_random_seed(
    0
)

global_desc_2, logits, rho_coords2 = MaSIF_ligand2(
    max_rho=params["max_distance"],
    n_ligands=params["n_classes"],
    feat_mask=params["feat_mask"],
    n_rotations=2
)([input_feat,rho_coords,theta_coords,mask,pocket_labels,tf.constant(1.0)])

logits

In [ ]:
# global_desc_1= tf.matmul(
#                     tf.transpose(global_desc_2), global_desc_2
#                 ) / tf.cast(tf.shape(global_desc_2)[0], tf.float32)
global_desc_1 = tf.reshape(global_desc_2, [1, -1])
print(global_desc_1.numpy().sum())
global_desc_1 = tf.nn.dropout(global_desc_1, 0)
print(global_desc_1.numpy().sum())
global_desc_1 = tf.keras.layers.Dense(
                    # self.global_desc_1, 
                    64, activation=tf.nn.relu
                )(global_desc_1)
print(global_desc_1.numpy().sum())
logits = tf.keras.layers.Dense(
                    # self.global_desc_1, 
                    10, activation=tf.identity
                )(global_desc_1)
logits.numpy()

In [ ]:
tf.compat.v1.set_random_seed(1)
W_conv = []
# initializer=tf.keras.initializers.glorot_normal(seed=10)
for i in range(5):
    
    W_conv.append(
            glorot_uniform((80, 80), seed=(i+1)*100)
                        
    )
[i.sum() for i in W_conv]  

In [ ]:
tf.compat.v1.set_random_seed(1)
W_conv = []
initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=10)
# initializer=tf.keras.initializers.glorot_normal(seed=10)
for i in range(5):
    W_conv.append(
           initializer(shape=(80, 80))
    )
[i.numpy().sum() for i in W_conv]

In [ ]:
params["n_classes"]=10

layer = MaSIF_ligand2(
    max_rho=params["max_distance"],
    n_ligands=params["n_classes"],
    feat_mask=params["feat_mask"]
)
# _ = layer(ops.zeros((1, 1)))

In [ ]:
from masif_modules.read_ligand_tfrecords import _parse_function

params = masif_opts["ligand"]

# Load dataset
training_data = tf.data.TFRecordDataset(
    os.path.join(params["tfrecords_dir"], "training_data.tfrecord")
)
validation_data = tf.data.TFRecordDataset(
    os.path.join(params["tfrecords_dir"], "validation_data.tfrecord")
)
testing_data = tf.data.TFRecordDataset(
    os.path.join(params["tfrecords_dir"], "testing_data.tfrecord")
)
training_data = training_data.map(_parse_function)
validation_data = validation_data.map(_parse_function)
testing_data = testing_data.map(_parse_function)

tf.random.set_seed(0)
training_iterator = iter(training_data)
# training_iterator=training_data.make_one_shot_iterator()
# data_element = training_iterator.get_next()
# data_element[5]

In [ ]:
from collections import OrderedDict
import pymesh
import numpy as np
import sys
test = [x.rstrip().split('.')[0] for x in open("feat.txt").readlines()]
# training_list = [x.rstrip() for x in open(params['training_list']).readlines()]


In [ ]:
my_dict = {}
len(test)

In [ ]:
f

In [ ]:
"data_preparation/01-benchmark_surfaces/"+f+"_protein.ply"

In [ ]:
from collections import OrderedDict
import pymesh
import numpy as np
import sys

test = [x.rstrip().split('.')[0] for x in open("names.txt").readlines()]
my_dict = {}

for i, f in enumerate(test[3300:3600]):
    if i % 10 == 0:
        print(i,f)
        sys.stdout.flush()
    m=pymesh.load_mesh("data_preparation/01-benchmark_surfaces/"+f+"_protein.ply")
    n=np.load("data_preparation/01-benchmark_surfaces/"+f+"_names.npy",
        allow_pickle=True).item()['names1']

    one = n[m.get_attribute("vertex_iface")==1]
    if len(one) > 0:
        if len(one.shape) > 1:
            one = np.concatenate(one)
        one = ['_'.join(np.array(i.split('_'))[1:4]) for i in one]
        if '384_x_TYR' in one:
            print(f)
        for name in one:
            if name in my_dict:
                my_dict[name]['1'] +=1
            else:
                my_dict[name] = {}
                my_dict[name]['1'] = 1
            if '2' not in my_dict[name]:
                my_dict[name]['2'] = 0
    two = n[m.get_attribute("vertex_iface")==2]
    if len(two) > 0:
        two = ['_'.join(np.array(i.split('_'))[1:4]) for i in np.concatenate(two)]
        for name in two:
            if name in my_dict:
                my_dict[name]['2'] +=1
            else:
                my_dict[name] = {}
                my_dict[name]['2'] = 1
        
res = OrderedDict(sorted(my_dict.items(),
       key = lambda x: x[1]['2'], reverse=True))
np.save("two.npy", res)
res = OrderedDict(sorted(my_dict.items(),
       key = lambda x: x[1]['1'], reverse=True))
np.save("one.npy", res)


In [ ]:
my_dict

In [ ]:
from collections import OrderedDict
res = OrderedDict(sorted(my_dict.items(),
       key = lambda x: x[1]['2'], reverse=True))
res
np.save("two.npy", res)

In [ ]:
res=np.load("one.npy", allow_pickle=True).item()
res

In [ ]:
dict(sorted(my_dict.items(), key = lambda x: x['1']))

In [ ]:
my_dict

In [ ]:
import pymesh
mesh=pymesh.load_mesh("data_preparation/01-benchmark_surfaces/Sim_1_traj_10_frame_1282_protein.ply")
(mesh.get_attribute('vertex_iface')==1).sum()


In [ ]:
precom_dir = params["masif_precomputation_dir"]
ligand_coord_dir = params["ligand_coords_dir"]
tfrecords_dir = params["tfrecords_dir"]
params["n_classes"]=10

tf.random.set_seed(0)
for num_train_sample in range(1):
# for num_train_sample in range(int(num_training_samples / 10)):
    try:
        # data_element = training_iterator.get_next()
        data_element = list(training_iterator)[0]
    except Exception as e:
        print(e)
    labels = data_element[4]
    print(data_element[5])
    n_ligands = labels.shape[1]
    # Choose a random ligand from the structure
    all_ligand_types = np.load(
        os.path.join(
            ligand_coord_dir, "{}_ligtypes.npy".format(data_element[5].numpy().decode("utf-8"))
        )
    ).astype(str)
    # random_ligand = np.random.choice(n_ligands, 1) # TODO: don't choose cofactor?
    
    random_ligand = np.random.choice(n_ligands-1, 1)+1 # TODO: don't choose cofactor?
    pocket_points = np.where(labels.numpy()[:, random_ligand] != 0.0)[0]
    label = np.max(labels.numpy()[:, random_ligand]) - 1
    pocket_labels = np.zeros(params["n_classes"], dtype=np.float32)
    pocket_labels[label] = 1.0
    npoints = pocket_points.shape[0]
    if npoints < 32:
        print('less than 32:', npoints, data_element[5], all_ligand_types[random_ligand])
        continue
    else:
        print(npoints, data_element[5], all_ligand_types[random_ligand])

    input_feat=data_element[0].numpy()[pocket_points[:32], :, :]
    rho_coords=np.expand_dims(data_element[1], -1)[pocket_points[:32], :, :]
    theta_coords=np.expand_dims(data_element[2], -1)[pocket_points[:32], :, :]
    mask=data_element[3].numpy()[pocket_points[:32], :, :]

In [ ]:
input_feat.shape

In [ ]:
# feed_dict = {
#         'Input_feat': input_feat,
#         'rho_coords':rho_coords,
#         'theta_coords': theta_coords,
#         'mask': mask,
#         'labels': pocket_labels,
#         'keep_prob': 1.0
#     }
# feed_dict = [input_feat,rho_coords,theta_coords,mask,pocket_labels,1.0]

global_desc_1, rho_coords2 = MaSIF_ligand2(
    max_rho=params["max_distance"],
    n_ligands=params["n_classes"],
    feat_mask=params["feat_mask"],
    n_rotations=2
)([input_feat,rho_coords,theta_coords,mask,pocket_labels,tf.constant(1.0)])

In [ ]:


batch_size=200
params["n_conv_layers"]=1

ppi_pair_id="Sim_1_traj_42_frame_1307"
mydir="data_preparation/04a-precomputation_12A/precomputation/" + ppi_pair_id + "/"
pdbid = ppi_pair_id.split(".")[0]
pid=pdbid
rho_wrt_center = np.load(mydir + pid + "_rho_wrt_center.npy")
theta_wrt_center = np.load(mydir + pid + "_theta_wrt_center.npy")
input_feat = np.load(mydir + pid + "_input_feat.npy")

iface_labels = np.load(mydir + pid + "_iface_labels.npy")

mask = np.load(mydir + pid + "_mask.npy")
mask = np.expand_dims(mask, 2)
indices = np.load(mydir + pid + "_list_indices.npy", allow_pickle=True).item()['protein']

# indices is (n_verts x <30), it should be
# indices = pad_indices(indices, mask.shape[1])
tmp = np.zeros((len(iface_labels), 3))
for i in range(len(iface_labels)):
    if iface_labels[i] == 1:
        tmp[i, 1] = 1
    elif iface_labels[i] == 2:
        tmp[i, 2] = 1
    else:
        tmp[i, 0] = 1
iface_labels_dc = tmp
super_pos = np.where(iface_labels == 2)[0]
pos_labels = np.where(iface_labels == 1)[0]
neg_labels = np.where(iface_labels == 0)[0]
print("PNS",len(pos_labels),len(neg_labels), len(super_pos))


n = min(len(pos_labels), len(neg_labels))
n = min(n, batch_size // 2)
subset = np.concatenate([neg_labels[:(n*5)], pos_labels[:n], super_pos])
print(len(subset))

In [ ]:
for num_train_sample in range(1):
# for num_train_sample in range(int(num_training_samples / 10)):
    try:
        data_element = sess.run(training_next_element)
    except Exception as e:
        print(e)
    labels = data_element[4]
    n_ligands = labels.shape[1]
    # Choose a random ligand from the structure
    all_ligand_types = np.load(
        os.path.join(
            ligand_coord_dir, "{}_ligtypes.npy".format(data_element[5].decode("utf-8"))
        )
    ).astype(str)
    random_ligand = np.random.choice(n_ligands, 1) # TODO: don't choose cofactor?
    pocket_points = np.where(labels[:, random_ligand] != 0.0)[0]
    label = np.max(labels[:, random_ligand]) - 1
    pocket_labels = np.zeros(params["n_classes"], dtype=np.float32)
    pocket_labels[label] = 1.0
    npoints = pocket_points.shape[0]
    if npoints < 32:
        print('less than 32:', npoints, data_element[5], all_ligand_types[random_ligand])
        continue
    else:
        print(npoints, data_element[5], all_ligand_types[random_ligand])

    input_feat=data_element[0][pocket_points[:32], :, :]
    rho_coords=np.expand_dims(data_element[1], -1)[pocket_points[:32], :, :]
    theta_coords=np.expand_dims(data_element[2], -1)[pocket_points[:32], :, :]
    mask=data_element[3][pocket_points[:32], :, :]

In [ ]:
from masif_modules.read_ligand_tfrecords import _parse_function
# Load dataset
training_data = tf.data.TFRecordDataset(
    os.path.join(params["tfrecords_dir"], "training_data.tfrecord")
)
training_data = training_data.map(_parse_function)
# training_iterator = training_data.make_one_shot_iterator()
# training_next_element = training_iterator.get_next()

batched_dataset

In [ ]:
batched_dataset

In [ ]:
from input_output.save_ply import save_ply
save_ply("mesh_test"+".ply", mesh.vertices,
         mesh.faces, normals=n, 
         charges=mesh.get_attribute("vertex_charge"),
         normalize_charges=False, 
         hbond=mesh.get_attribute("vertex_hbond"), 
         hphob=mesh.get_attribute("vertex_hphob"),
         iface=mesh.get_attribute("vertex_iface"))

In [ ]:
from masif_modules.read_data_from_surface import read_data_from_surface, compute_shape_complementarity
import os
import sys
import re
import numpy as np

params = masif_opts['site']
masif_opts["ply_file_template"] = masif_opts["ply_chain_dir"] + "/{}_{}.ply"

print(masif_opts['ply_file_template'])
# plane 7037
ppi_pair_list=['1A0H_D']
for ppi_pair_id in ppi_pair_list:
    print(ppi_pair_id)
    all_list_desc = []
    all_list_coords = []
    all_list_shape_idx = []
    all_list_names = []
    idx_positives = []
    # fields=ppi_pair_id.split('.')
    fields=ppi_pair_id.split('_')

    my_precomp_dir = params['masif_precomputation_dir']
    if not os.path.exists(my_precomp_dir):
        os.makedirs(my_precomp_dir)

    ply_file = {}
    ply_file['p1'] = masif_opts['ply_file_template'].format(fields[0], fields[1])
    if len (fields) == 2 or fields[2] == '':
        pids = ['p1']
    
    rho = {}
    neigh_indices = {}
    mask = {}
    input_feat = {}
    theta = {}
    iface_labels = {}
    verts = {}

    pid=pids[0]
    print(ply_file[pid])
    (input_feat[pid], 
     rho[pid], 
     theta[pid], 
     mask[pid], 
     neigh_indices[pid], 
     iface_labels[pid], 
     verts[pid]) = read_data_from_surface(ply_file[pid], 
                                          params, 
                                          fields[0])

In [ ]:
pdb='Sim_1_traj_10_frame_1236'
regular_mesh=pymesh.load_mesh('data_preparation/01-benchmark_surfaces/%s_protein.ply' % pdb)
regular_mesh.get_attribute("vertex_iface").sum()

# full_regular_mesh=pymesh.load_mesh('data_preparation/01-benchmark_surfaces/%s_protein_complex.ply' % pdb)
# full_regular_mesh.get_attribute("vertex_iface").sum()

regular_names = np.load('data_preparation/01-benchmark_surfaces/%s_names.npy' % pdb, allow_pickle=True).item()

full_regular_names = np.load('data_preparation/01-benchmark_surfaces/%s_fullnames.npy' % pdb, allow_pickle=True).item()

regular_names=np.concatenate(regular_names['names1'])
full_regular_names=full_regular_names['names1']
print(len(full_regular_names), len(regular_names))

In [ ]:
# Neural network ligand application specific parameters.
import os
import sys
from multiprocessing import Pool
import subprocess as sp
import itertools

sys.path.insert(0, "/data/pompei/bw973/Oxygenases/masif/source")
from default_config.masif_opts import masif_opts
masif_opts["ply_file_template"] = masif_opts["ply_chain_dir"] + "/{}_protein.ply"
masif_opts["ligand"] = {}
masif_opts["ligand"]["assembly_dir"] = "data_preparation/00b-pdbs_assembly/"
masif_opts["ligand"]["assembly_dir2"] = "data_preparation/00c-pdbs_assembly/"
masif_opts["ligand"]["ligand_coords_dir"] = "data_preparation/00c-ligand_coords/"
masif_opts["ligand"][
    "masif_precomputation_dir"
] = "data_preparation/04a-precomputation_12A/precomputation/"
masif_opts["ligand"]["max_shape_size"] = 50
masif_opts["ligand"]["feat_mask"] = [1.0] * 5
masif_opts["ligand"]["train_fract"] = 0.9 * 0.8
masif_opts["ligand"]["val_fract"] = 0.1 * 0.8
masif_opts["ligand"]["test_fract"] = 0.2
masif_opts["ligand"]["tfrecords_dir"] = "data_preparation/tfrecords/"
masif_opts["ligand"]["max_distance"] = 5.0
masif_opts["ligand"]["n_classes"] = 7
masif_opts["ligand"]["feat_mask"] = [1.0, 1.0, 1.0, 1.0, 1.0]
masif_opts["ligand"]["costfun"] = "dprime"
masif_opts["ligand"]["model_dir"] = "nn_models/all_feat/"
masif_opts["ligand"]["test_set_out_dir"] = "test_set_predictions/"

def convert_to_string(binary):
    return binary.decode('utf-8')

def _run_command(cmd_info):
    """Helper function for submitting commands parallelized."""
    cmd, supress = cmd_info
    p = sp.Popen(cmd, shell=True, stdout=sp.PIPE, stderr=sp.PIPE)
    output, err = p.communicate()
    if convert_to_string(err) != '' and not supress:
        print("\nERROR: " + convert_to_string(err))
        raise
    output = convert_to_string(output)
    p.terminate()
    return output


def run_commands(cmds, supress=False, n_procs=1):
    """Wrapper for submitting commands to shell"""
    if type(cmds) is str:
        cmds = [cmds]
    if n_procs == 1:
        outputs = []
        for cmd in cmds:
            outputs.append(_run_command((cmd, supress)))
    else:
        cmd_info = list(zip(cmds, itertools.repeat(supress)))
        pool = Pool(processes = n_procs)
        outputs = pool.map(_run_command, cmd_info)
        pool.terminate()
    return outputs

from multiprocessing import Pool

# pool = Pool(processes=5)
# _ = pool.map(preprocess, range(1,6))

In [ ]:
# need to preprocess to only select O2 within 3.5 angstroms of protein
import mdtraj as md
import numpy as np
import scipy
from subprocess import Popen, PIPE
import subprocess as sp
from biopandas.pdb import PandasPdb
import sys

# which frames have O2IF within 2.5 angstroms


def cmap_per_residue(traj,sel_list,ref_list,cutoff=3.5,metric='euclidean'):
    if len(ref_list)==0 or len(sel_list)==0:
        print("One of the selections is empty")
        return []
    coord1 = traj.xyz[:,sel_list,:]*10.0 # gas
    coord2 = traj.xyz[:,ref_list,:]*10.0 # protein
    residue_sel = np.array([traj.topology.atom(ind).residue.index for ind in sel_list]) # gas
    residue_ref = np.array([traj.topology.atom(ind).residue.index for ind in ref_list])
    residue_sel_un = np.unique(residue_sel) # gas
    residue_ref_un = np.unique(residue_ref)
    dic_sel={t:c  for c,t in enumerate(residue_sel_un)} # gas
    dic_ref={t:c  for c,t in enumerate(residue_ref_un)}
    histo = np.zeros((len(coord1),len(residue_sel_un),len(residue_ref_un)))
    for c,j in enumerate(coord1):
        axis_sel,axis_ref = np.where(scipy.spatial.distance.cdist(coord1[c],coord2[c],metric=metric) < cutoff)
        real_a= residue_sel[axis_sel]
        real_b= residue_ref[axis_ref]
        elements,counts = np.unique(list(zip(real_a,real_b)),axis=0,return_counts=True)
        for c1,el in enumerate(elements):
            ax1=dic_sel[el[0]];
            ax2=dic_ref[el[1]];
            histo[c,ax1,ax2] = counts[c1];
    return residue_sel_un,residue_ref_un,histo

# gas = sys.argv[1]
gas = 'O2IF'
gases_ligands = ['O2I','N2I','O2Q','N2']#, 'AKG'
simnum=1
trjnum=1
# top = '/data/pompei/bw973/Oxygenases/PHD2/PHD2_50_O2IF/Bundle/Sim%d/equil5.gro' % simnum
nfrcond=1
lengths = []
top = 'equil5.gro'
# trjconv -f $i -s run1.tpr -o an$number.xtc -pbc mol -ur compact -center <<< "14 0"
# for trjnum in range(1, 5):
def preprocess(trjnum, gas='O2IF',check=False):
    print(trjnum)
    cmds = ["echo '14 0' | gmx trjconv -f run%d -s run1.tpr -o an%d.xtc -pbc mol -ur compact -center" % (trjnum,trjnum)]
    if check:
        if not os.path.exists("an%d.xtc" % trjnum):
            _ = run_commands(cmds, supress=True)
        else:
            print("an%d.xtc exists" % trjnum)
    else:
        print("making an%d.xtc" % trjnum)
        _ = run_commands(cmds, supress=True)
    # args = ["gmx", "trjconv", "-f", 'run%d.xtc' % (trjnum), "-pbc", "mol", "-ur",\
    #      "compact", "-center", '-o', 'an%d.xtc' % (trjnum), "-s", "run1.tpr" ,  "<<<", '14 0']
    # p2 = Popen(args, stdout=PIPE, stderr=PIPE)
    # stdout, stderr = p2.communicate()
    t = '/data/pompei/bw973/Oxygenases/PHD2/PHD2_50_O2IF/Bundle/Sim%d/an%d.xtc' % (simnum,trjnum)
    print("loading centered an%d.xtc" % trjnum)
    traj = md.load(t, top=top)
    time=np.arange(len(traj))*.01
    protein = traj.topology.select('protein')
    gas2 = traj.topology.select('resname %s' % gas)
    AKG = traj.topology.select('resname AKG')

    test = cmap_per_residue(traj,gas2,protein,cutoff=2.5)
    test_akg = cmap_per_residue(traj,gas2,AKG,cutoff=3.87)
    segments_dict = {}
    for i in range(0,24):
        res=test[2][0:,i]
        t=np.zeros(len(traj))
        res_akg=test_akg[2][0:,i]
        
        t[np.concatenate((np.where(res)[0], np.where(res_akg)[0]))] = 1

        segment=[]
        subsegment,subtime,subframe=[],[],[]
        for c,value in enumerate(t):
            if value==0 and len(subsegment)==0:
                subsegment,subtime=[],[]
            elif value==0 and len(subsegment)!=0:
                if len(subsegment) > nfrcond: 
                    segment.append(np.array([subframe,subtime,subsegment]))
                subsegment,subtime,subframe=[],[],[]
            else: 
                subsegment.append(value)
                subtime.append(time[c])
                subframe.append(c)
        if len(subsegment) > nfrcond: segment.append(np.array([subframe,subtime,subsegment]))
        segments_dict[i] = segment

    l=[]
    for k in segments_dict.keys():
        if len(segments_dict[k]) > 0:
            sl = np.concatenate([r[0] for r in segments_dict[k]])
            l.append(np.unique(sl))
    if len(l) > 0:
        frames = np.unique(np.concatenate(l).astype(int))
        fn = frames+1
        print(len(fn))
        lengths.append(len(fn))
    else:
        print(0)
        lengths.append(0)
    np.save('lengths2.npy', lengths)
    trajf=traj[frames]
    print(np.array(frames))

    for i,f in enumerate(frames):
        trajfn = trajf[i]
        l2 = np.array([], dtype=np.int32)
        test2 = cmap_per_residue(trajfn,gas2,protein,cutoff=2.5)
        test2_akg = cmap_per_residue(trajfn,gas2,AKG,cutoff=5.5)
        whr = np.where([j.sum()>0 for j in test2[2][0]])[0]*2
        whr_akg = np.where([j.sum()>0 for j in test2_akg[2][0]])[0]*2
        l2 = np.append(l2, whr)
        l2 = np.append(l2, whr_akg)
        # print(i,f,l2)
        # break
        whr2 = np.concatenate([(n,(n+1)) for n in np.unique(l2)])
        # print(whr2)
        atms = np.sort(np.concatenate((gas2[whr2],AKG,protein)))
        sel = trajfn.atom_slice(atms)
        # atms = np.sort(np.concatenate((gas2[whr2],AKG,protein)))
        # sel = trajfn.atom_slice(atms)
        pdb="Sim_%d_traj_%d_frame_%d" % (simnum, trjnum, f+1)+'.pdb'
        sel.save(masif_opts["ligand"]["assembly_dir"]+'/'+pdb)
        
        ppdb_df = PandasPdb().read_pdb(masif_opts["ligand"]["assembly_dir"]+'/'+pdb)
        o2if = ppdb_df.df['ATOM']
        o2if.loc[o2if.residue_name=='O2I','atom_name'] = np.concatenate([['OI1','OI2']] * (len(o2if.loc[o2if.residue_name=='O2I','atom_name'])//2))
        ppdb_df._df['ATOM'] = o2if[(o2if['residue_name']!='ACE') & (o2if['residue_name']!='HOH')]
        atom = o2if[~o2if.residue_name.isin(gases_ligands)]
        hetatm = o2if[o2if.residue_name.isin(gases_ligands)]
        ppdb_df._df['ATOM'] = atom
        ppdb_df._df['HETATM'] = hetatm
        ppdb_df._df['HETATM'].loc[:,'record_name']='HETATM'
        ppdb_df.to_pdb(masif_opts["ligand"]["assembly_dir"]+'/'+pdb)

        df_atom=ppdb_df._df['ATOM']
        df=ppdb_df._df['HETATM']
        o2i_uniq = np.unique(df[df.residue_name=='O2I'].residue_number)
        o2i_uniq = np.sort(o2i_uniq)

        lig_types = []
        akg_rn = np.unique(df_atom[df_atom.residue_name=='AKG'].residue_number)
        lig_types.append(['AKG','res_%d' % akg_rn])
        [lig_types.append([gas, 'res_%d' % i]) for i in o2i_uniq]
        np.save(masif_opts["ligand"]["ligand_coords_dir"]+pdb.split('.')[0]+'_ligtypes.npy', lig_types)

        lig_coords = {}
        lig_coords[akg_rn[0]] = df[df.residue_name=='AKG'][['x_coord', 'y_coord','z_coord']].values
        for val in o2i_uniq:
            # print(val)
            lig_coords[val] = df[df.residue_number==val][['x_coord', 'y_coord','z_coord']].values

        np.save(masif_opts["ligand"]["ligand_coords_dir"]+pdb.split('.')[0]+'_ligcoords.npy', lig_coords)
    cmds = ["rm an%d.xtc" % (trjnum)]
    run_commands(cmds, supress=True)
# preprocess(trjnum)

import sys
from functools import partial
# python get_close_pdbs.py 2 3 O2IF 2
# processes = int(sys.argv[1])
# rng = int(sys.argv[2])
# pool = Pool(processes=processes)
# gas = sys.argv[3]
# _ = pool.map(partial(preprocess, gas=gas), range(1,1))

In [ ]:
top = 'equil5.gro'
t = '/data/pompei/bw973/Oxygenases/PHD2/PHD2_50_O2IF/Bundle/Sim%d/an%d.xtc' % (simnum,trjnum)
print("loading centered an%d.xtc" % trjnum)
traj = md.load(t, top=top)
time=np.arange(len(traj))*.01
protein = traj.topology.select('protein')
gas2 = traj.topology.select('resname %s' % gas)
AKG = traj.topology.select('resname AKG')

test = cmap_per_residue(traj,gas2,protein,cutoff=2.5)
test_akg = cmap_per_residue(traj,gas2,AKG,cutoff=3.87)
segments_dict = {}
for i in range(0,24):
    res=test[2][0:,i]
    t=np.zeros(len(traj))
    res_akg=test_akg[2][0:,i]
    
    t[np.concatenate((np.where(res)[0], np.where(res_akg)[0]))] = 1

    segment=[]
    subsegment,subtime,subframe=[],[],[]
    for c,value in enumerate(t):
        if value==0 and len(subsegment)==0:
            subsegment,subtime=[],[]
        elif value==0 and len(subsegment)!=0:
            if len(subsegment) > nfrcond: 
                segment.append(np.array([subframe,subtime,subsegment]))
            subsegment,subtime,subframe=[],[],[]
        else: 
            subsegment.append(value)
            subtime.append(time[c])
            subframe.append(c)
    if len(subsegment) > nfrcond: segment.append(np.array([subframe,subtime,subsegment]))
    segments_dict[i] = segment

l=[]
for k in segments_dict.keys():
    if len(segments_dict[k]) > 0:
        sl = np.concatenate([r[0] for r in segments_dict[k]])
        l.append(np.unique(sl))
if len(l) > 0:
    frames = np.unique(np.concatenate(l).astype(int))
    fn = frames+1
    print(len(fn))
    lengths.append(len(fn))
else:
    print(0)
    lengths.append(0)
np.save('lengths2.npy', lengths)
trajf=traj[frames]
print(np.array(frames))

In [ ]:
i,f=list(enumerate(frames))[14]
trajfn = trajf[i]
l2 = np.array([], dtype=np.int32)
test2 = cmap_per_residue(trajfn,gas2,protein,cutoff=2.5)
test2_akg = cmap_per_residue(trajfn,gas2,AKG,cutoff=5.5)
whr = np.where([j.sum()>0 for j in test2[2][0]])[0]*2
whr_akg = np.where([j.sum()>0 for j in test2_akg[2][0]])[0]*2
l2 = np.append(l2, whr)
l2 = np.append(l2, whr_akg)
# print(i,f,l2)
# break
whr2 = np.concatenate([(n,(n+1)) for n in np.unique(l2)])
# print(whr2)
atms = np.sort(np.concatenate((gas2[whr2],AKG,protein)))
sel = trajfn.atom_slice(atms)
# atms = np.sort(np.concatenate((gas2[whr2],AKG,protein)))
# sel = trajfn.atom_slice(atms)
pdb="Sim_%d_traj_%d_frame_%d" % (simnum, trjnum, f+1)+'.pdb'
sel.save(masif_opts["ligand"]["assembly_dir"]+'/'+pdb)

In [ ]:
ppdb_df = PandasPdb().read_pdb(masif_opts["ligand"]["assembly_dir"]+'/'+'Sim_1_traj_1_frame_190.pdb')
o2if = ppdb_df.df['ATOM']
# o2if.loc[o2if.residue_name=='O2I','atom_name'] = np.concatenate([['OI1','OI2']] * (len(o2if.loc[o2if.residue_name=='O2I','atom_name'])//2))
# ppdb_df._df['ATOM'] = o2if[(o2if['residue_name']!='ACE') & (o2if['residue_name']!='HOH')]
atom = o2if[~o2if.residue_name.isin(gases_ligands)]
hetatm = o2if[o2if.residue_name.isin(gases_ligands)]
ppdb_df._df['ATOM'] = atom
ppdb_df._df['HETATM'] = hetatm
ppdb_df._df['HETATM'].loc[:,'record_name']='HETATM'
ppdb_df.to_pdb(masif_opts["ligand"]["assembly_dir"]+'/'+pdb)

df_atom=ppdb_df._df['ATOM']
df=ppdb_df._df['HETATM']
o2i_uniq = np.unique(df[df.residue_name=='O2I'].residue_number)
o2i_uniq = np.sort(o2i_uniq)

lig_types = []
akg_rn = np.unique(df_atom[df_atom.residue_name=='AKG'].residue_number)
lig_types.append(['AKG','res_%d' % akg_rn])
[lig_types.append([gas, 'res_%d' % i]) for i in o2i_uniq]
# np.save(masif_opts["ligand"]["ligand_coords_dir"]+pdb.split('.')[0]+'_ligtypes.npy', lig_types)

lig_coords = {}
lig_coords[akg_rn[0]] = df_atom[df_atom.residue_name=='AKG'][['x_coord', 'y_coord','z_coord']].values

In [ ]:
pdbs = glob.glob(masif_opts["ligand"]["assembly_dir"]+'/*.pdb')
# pdbs = [i.split("/")[2] for i in pdbs]
# pdbs
# np.save(masif_opts["ligand"]["ligand_coords_dir"]+pdb.split('.')[0]+'_ligcoords.npy', lig_coords)

In [ ]:
pdb='data_preparation/00b-pdbs_assembly/Sim_1_traj_7_frame_1669.pdb'

In [ ]:
for pdb in pdbs[10:]:
# for pdb in ['data_preparation/00b-pdbs_assembly/Sim_1_traj_1_frame_190.pdb']:
    # print(pdb)
    pref=pdb.split("/")[2].split('.')[0]
    print(pref)
    ppdb_df = PandasPdb().read_pdb(pdb)
    df_atom=ppdb_df._df['ATOM']
    lig_coords = np.load(masif_opts["ligand"]["ligand_coords_dir"]+pref+'_ligcoords.npy',
                         allow_pickle=True).item()
    akg_rn = np.unique(df_atom[df_atom.residue_name=='AKG'].residue_number)
    lig_coords[akg_rn[0]] = df_atom[df_atom.residue_name=='AKG'][['x_coord', 'y_coord','z_coord']].values
    # print(lig_coords[akg_rn[0]])
    np.save(masif_opts["ligand"]["ligand_coords_dir"]+pref+'_ligcoords.npy', lig_coords)

In [ ]:
lig_coords

In [ ]:
import pymesh
import numpy as np

# test_mesh = pymesh.load_mesh('/data/pompei/bw973/Oxygenases/masif/data/masif_ligand/data_preparation/01-benchmark_surfaces/1IQP_ACBEDF.ply')
test_mesh = pymesh.load_mesh('data_preparation/01-benchmark_surfaces/Sim_2_traj_50_frame_68_protein.ply')
test_mesh.get_attribute("vertex_iface").sum()

In [ ]:
regular_mesh=pymesh.load_mesh('data_preparation/01-benchmark_surfaces/%s_protein.ply' % pdb)
regular_mesh.get_attribute("vertex_iface").sum()

In [ ]:
pdb='Sim_1_traj_10_frame_1556'
regular_mesh=pymesh.load_mesh('data_preparation/01-benchmark_surfaces/%s_protein.ply' % pdb)
regular_mesh.get_attribute("vertex_iface").sum()

full_regular_mesh=pymesh.load_mesh('/tmp/%s_protein_complex.ply' % pdb)
full_regular_mesh.get_attribute("vertex_iface").sum()

regular_names = np.load('data_preparation/01-benchmark_surfaces/%s_names.npy' % pdb, allow_pickle=True).item()

full_regular_names = np.load('data_preparation/01-benchmark_surfaces/%s_fullnames.npy' % pdb, allow_pickle=True).item()

regular_names=regular_names['names1']
full_regular_names=full_regular_names['names1']
print(len(full_regular_mesh.vertices), len(regular_names))

In [ ]:
regular_names['names1']

In [ ]:
from triangulation.computeMSMS import computeMSMS, computeMSMS2
probe='0.70'
vertices1, faces1, normals1, names1, areas1 = computeMSMS2("/tmp/Sim_1_traj_10_frame_1556_protein.pdb",
        protonate=True, probe=probe)

In [ ]:
np.where([re.search('AKG',i) for i in regular_names])[0]

In [ ]:
full_regular_names

vertices2 = vertices1
faces2 = faces1
from triangulation.computeCharges import computeCharges, assignChargesToNewMesh, assignAtomNamesToNewMesh
from triangulation.fixmesh import fix_mesh
# Step 4: Fix the mesh.
mesh = pymesh.form_mesh(vertices2, faces2)
regular_mesh = fix_mesh(mesh, 0.8)

regular_names = assignAtomNamesToNewMesh(regular_mesh.vertices, 
                                          vertices1, names1) 
regular_names=np.concatenate(regular_names)    
np.where([re.search('AKG',i) for i in regular_names])[0]

In [ ]:
len(A)

In [ ]:
# (regular_mesh.get_attribute("vertex_iface")==2).sum()
(regular_mesh.get_attribute("vertex_iface")==1).sum()
# (regular_mesh.get_attribute("vertex_iface")).sum()
mesh=full_regular_mesh
import re
import scipy

In [ ]:
full_iface = np.zeros(len(full_regular_names))
full_iface[np.where([re.search('O2I',i) for i in full_regular_names])[0]] = 1

 # vertices from full mesh with gas/everything
O = np.where([re.search('O2I',i) for i in full_regular_names])[0]
tO=mesh.vertices[O]
# vertices from regular mesh of protein/cofactor
A=np.where([re.search('AKG',i) for i in regular_names])[0]
tA=regular_mesh.vertices[A]

t2=scipy.spatial.distance_matrix(regular_mesh.vertices,tO)
t3=scipy.spatial.distance_matrix(regular_mesh.vertices,tA)
iface1=np.where([(i<3).sum() for i in t2])[0] # protein res near gas
iface2=np.where([(i<5).sum() for i in t3])[0] # protein res near cofactor
iface3=np.where([(i<2).sum() for i in t2])[0] # protein res VERY near gas

In [ ]:
A

In [ ]:
import scipy

t2=scipy.spatial.distance_matrix(regular_mesh.vertices,tO)
# np.where([(i<1).sum() for i in t2])
iface1=regular_names[np.where([(i<3).sum() for i in t2])]
iface1=np.where([(i<3).sum() for i in t2])[0]
len(iface1)
# regular_names

In [ ]:
regular_mesh.get_attribute("vertex_cofactor_dist").sum()

In [ ]:
dist_near_gas_to_AKG[iface1].sum()

In [ ]:
t3=scipy.spatial.distance_matrix(regular_mesh.vertices,tA)
# np.where([(i<1).sum() for i in t2])

iface2=np.where([(i<5).sum() for i in t3])[0]
len(iface2)
regular_names[np.where([(i<2).sum() for i in t3])]

In [ ]:
regular_names = np.load('data_preparation/01-benchmark_surfaces/%s_names.npy' % pdb, allow_pickle=True).item()['names1']


In [ ]:
from masif_modules.read_data_from_surface2 import read_data_from_surface, compute_shape_complementarity
import pymesh
from sklearn.neighbors import KDTree
import numpy as np
from triangulation.fixmesh import fix_mesh
from default_config.masif_opts import masif_opts


params = masif_opts['ligand']



sufs = ['protein','gas']

# pdb=sys.argv[1]
pdb='Sim_1_traj_10_frame_1042'
fields = pdb.split('.')
pid=fields[0]
# masif_app = sys.argv[2]

rho = {}
neigh_indices = {}
mask = {}
input_feat = {}
theta = {}
iface_labels = {}
verts = {}


# for suf in sufs:
    # ply_file = masif_opts["ply_chain_dir"] + "/{}_protein.ply".format(pid)
suf="protein"
p_ply_file = masif_opts["ply_chain_dir"] + "{}_{}.ply".format(pid,suf)
print(p_ply_file)
input_feat[suf], rho[suf], theta[suf], mask[suf], neigh_indices[suf], iface_labels[suf], verts[suf] = read_data_from_surface(p_ply_file, params, pid=pid+"_"+suf)
print(len(verts[suf]))

suf="gas"
g_ply_file = masif_opts["ply_chain_dir"] + "{}_{}.ply".format(pid,suf)
print(g_ply_file)
input_feat[suf], rho[suf], theta[suf], mask[suf], neigh_indices[suf], iface_labels[suf], verts[suf] = read_data_from_surface(g_ply_file, params, False, pid+"_"+suf)
print(len(verts[suf]))

# print(pid)
#         print(ply_file[pid])
#         input_feat[pid], rho[pid], theta[pid], mask[pid], neigh_indices[pid], iface_labels[pid], verts[pid] = read_data_from_surface(ply_file[pid], params, my_precomp_dir+pid)
#         print(len(verts[pid]))
#         pad=params["max_shape_size"]
#         neigh_indices[pid]= np.array([i + [-1]*(pad-len(i)) for i in neigh_indices[pid]])



# pid=pids[0]
# input_feat[pid], rho[pid], theta[pid], mask[pid], neigh_indices[pid], iface_labels[pid], verts[pid] = read_data_from_surface(ply_file[pid], params)

In [ ]:
regular_mesh=pymesh.load_mesh(p_ply_file)
regular_mesh.get_attribute("vertex_iface")==1

In [ ]:
[len(i) for i in neigh_indices[suf]]

In [ ]:
params = masif_opts['ppi_search']
params

In [ ]:
params = masif_opts['ligand']
params

In [ ]:
labels[regular_mesh.get_attribute("vertex_iface")==1]

In [ ]:
regular_names
mylabels = p1_sc_labels[0]
labels = np.median(mylabels, axis=1)
np.where((labels < 1) & (labels > 0.5))[0]
            

In [ ]:
my_precomp_dir+pid+'prot_sc_labels'

In [ ]:
params['sc_w']=0.015
params['sc_interaction_cutoff']=3.0
params['sc_radius']=params['max_distance']
my_precomp_dir = params['masif_precomputation_dir']

p1_sc_labels, p2_sc_labels = compute_shape_complementarity(p_ply_file, g_ply_file, neigh_indices['protein'],neigh_indices['gas'], rho['protein'], rho['gas'], mask['protein'], mask['gas'], params)
np.save(my_precomp_dir+pid+'_prot_sc_labels', p1_sc_labels)
np.save(my_precomp_dir+pid+'_gas_sc_labels', p2_sc_labels)
mylabels = p1_sc_labels[0]
labels = np.median(mylabels, axis=1)
print(labels.max())
labels[regular_mesh.get_attribute("vertex_iface")==1]

In [ ]:
(regular_mesh.get_attribute("vertex_iface")==1).sum()

In [ ]:
regular_names[pos_labels]

In [ ]:
mylabels = p1_sc_labels[0]
labels = np.median(mylabels, axis=1)
print(pid, labels.max())
labels[regular_mesh.get_attribute("vertex_iface")==1]


pos_labels = np.where(((labels < -0.2) & (labels > -0.6) | (labels > 0.1)))[0]
print(pid,"len(pos_labels)", len(pos_labels))
regular_names[pos_labels]

print(pid, "len_intersect_iface",len(np.intersect1d(
    np.where(regular_mesh.get_attribute("vertex_iface")==1),
    pos_labels
)))

In [ ]:
pos_labels = np.where(((labels < 0) & (labels > -0.65) | (labels > 0.65)))[0]
print(len(pos_labels))
regular_names[pos_labels]

In [ ]:
(regular_mesh.get_attribute("vertex_iface")==1).sum()

In [ ]:
(labels[regular_mesh.get_attribute("vertex_iface")==1]).shape

In [ ]:
p2_sc_labels[0].sum(axis=1)

In [ ]:
suf="protein"
ply_file = masif_opts["ply_chain_dir"] + "{}_{}.ply".format(pid,suf)
print(ply_file)

In [ ]:
test = np.load("/data/pompei/bw973/Oxygenases/masif/source/test/test.npy")

In [ ]:
ALPHA
import os
os.path.basename("data/pompei/bw973/Oxygenases/masif/source/test/test.npy")

In [ ]:
from pymol.cgo import *

obj = []
obj.extend([BEGIN, 0.5])
obj


In [ ]:
iface3=np.where([(i<2).sum() for i in t2])[0]
regular_names[np.intersect1d(iface3, iface2)]

In [ ]:
regular_names[np.intersect1d(iface3, iface2)]

In [ ]:
names=np.concatenate(np.load('data_preparation/01-benchmark_surfaces/Sim_2_traj_42_frame_1367_names.npy',
    allow_pickle=True).item()['names1'])
len(names)

In [ ]:
# Header variables and parameters.
import sys
import pymesh
import os
import numpy as np
from IPython.core.debugger import set_trace
import importlib

from scipy.spatial import cKDTree

from default_config.masif_opts import masif_opts

"""
masif_ppi_search_cache_training_data.py: Function to cache all the training data for MaSIF-search. 
                This function extract all the positive pairs and a random number of negative surfaces.
                In the future, the number of negative surfaces should be increased.
Pablo Gainza - LPDI STI EPFL 2019
Released under an Apache License 2.0
"""

params = masif_opts['ppi_search']

custom_params = {}
custom_params['cache_dir'] = 'nn_models/sc05/cache/'
custom_params['model_dir'] = 'nn_models/sc05/all_feat/model_data/'
custom_params['desc_dir'] = 'output/sc05/all_feat/model_data/'
custom_params['gif_eval_out'] = 'nn_models/sc05/gif_eval/'
custom_params['min_sc_filt'] = 0.5
custom_params['max_sc_filt'] = 1.0
custom_params['pos_surf_accept_probability'] = 1.0

for key in custom_params: 
    print('Setting {} to {} '.format(key, custom_params[key]))
    params[key] = custom_params[key]

if 'pids' not in params: 
    params['pids'] = ['p1', 'p2']


# Read the positive first 
parent_in_dir = params['masif_precomputation_dir']

binder_rho_wrt_center = []
binder_theta_wrt_center = []
binder_input_feat = []
binder_mask = []

pos_rho_wrt_center = []
pos_theta_wrt_center = []
pos_input_feat = []
pos_mask = []

neg_rho_wrt_center = []
neg_theta_wrt_center = []
neg_input_feat = []
neg_mask = []

np.random.seed(0)
training_idx = []
val_idx = []
test_idx = []
pos_names = []
neg_names = []

training_list = [x.rstrip() for x in open(params['training_list']).readlines()]
testing_list = [x.rstrip() for x in open(params['testing_list']).readlines()]

idx_count = 0
for count, ppi_pair_id in enumerate(os.listdir(parent_in_dir)[0:10]):
    if ppi_pair_id not in testing_list and ppi_pair_id not in training_list:
        continue
    in_dir = parent_in_dir + ppi_pair_id+'/'
    print(ppi_pair_id)

In [ ]:
ppi_pair_id="Sim_1_traj_42_frame_1246"
in_dir = parent_in_dir + ppi_pair_id+'/'

In [ ]:
labels = np.load(in_dir+ppi_pair_id+'_prot_sc_labels.npy')
# Take the median of the percentile 25 shape complementarity.
mylabels = labels[0]
labels = np.median(mylabels, axis=1)
pos_labels = np.where(((labels < -0.2) & (labels > -0.6) | (labels > 0.1)))[0]
K = int(params['pos_surf_accept_probability']*len(pos_labels))
l = np.arange(len(pos_labels))
np.random.shuffle(l)
l = l[:K] 
l = pos_labels[l]

In [ ]:
len(pos_labels)
ply_fn1=masif_opts['ply_file_template'].format(ppi_pair_id,"protein")
ply_fn2=masif_opts['ply_file_template'].format(ppi_pair_id,"gas")

In [ ]:
m=pymesh.load_mesh(ply_fn1)
m.get_attribute("vertex_iface").sum()


v0 = pymesh.load_mesh(ply_fn1).vertices
v1 = v0[l]
v2 = pymesh.load_mesh(ply_fn2).vertices

In [ ]:
import scipy


t2=scipy.spatial.distance_matrix(v0,v2)
names=np.load(masif_opts['ply_chain_dir']+ppi_pair_id+'_names.npy', allow_pickle=True).item()
len(names['names1'])

In [ ]:
print(len(names['names1'][m.get_attribute("vertex_iface")==1]))
names['names1'][m.get_attribute("vertex_iface")==2]

In [ ]:
test=[all(i>20) for i in t2]
print(sum(test))
names['names1'][test]

In [ ]:
# far from oxygen and far from cofactor

In [ ]:
len(l)

In [ ]:
contact_points = np.where(d < params['pos_interface_cutoff'])[0]
l[contact_points]

In [ ]:
rho_wrt_center = np.load(in_dir+ppi_pair_id+'_rho_wrt_center.npy')
rho_wrt_center.shape

In [ ]:
kdt = cKDTree(v1)
dneg, rneg = kdt.query(v2)
dneg

In [ ]:
print(len(dneg), len(v1), len(v2))

In [ ]:
kdt = cKDTree(v0)
dneg, rneg = kdt.query(v2)
dneg

In [ ]:
try:
    kdt = cKDTree(v1)
    dneg, rneg = kdt.query(v2)
except:
    set_trace()
k_neg2 = np.where(dneg > params['pos_interface_cutoff']*2)[0]
k_neg2

In [ ]:
np.where(d < 2)[0]

In [ ]:
kdt.query?

In [ ]:
kdt = cKDTree(v2)
d,r= kdt.query(v1)
# Contact points: those within a cutoff distance.
contact_points = np.where(d < params['pos_interface_cutoff'])[0]
try:
    k1 = l[contact_points]
except:
k2 = r[contact_points]

In [ ]:


v1 = pymesh.load_mesh(ply_fn1)
v2 = pymesh.load_mesh(ply_fn2)
# v1 = pymesh.load_mesh(ply_fn1).vertices[l]
# v2 = pymesh.load_mesh(ply_fn2).vertices

In [ ]:
# Read the corresponding ply files. 
fields = ppi_pair_id.split('_')
ply_fn1 = masif_opts['ply_file_template'].format(fields[0], fields[1])
ply_fn2 = masif_opts['ply_file_template'].format(fields[0], fields[2])

In [ ]:
regular_names = np.load('data_preparation/01-benchmark_surfaces/Sim_2_traj_42_frame_1410_names.npy',
    allow_pickle=True).item()

regular_mesh=pymesh.load_mesh('data_preparation/01-benchmark_surfaces/Sim_2_traj_42_frame_1410_protein.ply')
regular_mesh.get_attribute("vertex_iface").sum()

full_regular_mesh=pymesh.load_mesh('data_preparation/01-benchmark_surfaces/Sim_2_traj_42_frame_1410_protein_complex.ply')
full_regular_mesh.get_attribute("vertex_iface").sum()
                       

In [ ]:
regular_names['names1'][np.where([(i<4).sum() for i in t2])]

In [ ]:
np.where([re.search('A_210',i) for i in regular_names])[0]

In [ ]:
from sklearn.neighbors import KDTree

kdt = KDTree(v3)
d, r = kdt.query(regular_mesh.vertices)
d = np.square(d)

In [ ]:
np.where(d==max(d))

In [ ]:
d.max()
np.where(d>=0.5)

In [ ]:

test_names['names1'][np.where(d>=0.5)]

In [ ]:
from Bio.PDB import *

function
# Exclude disordered atoms.
class NotDisordered(Select):
    def accept_atom(self, atom):
        return not atom.is_disordered() or atom.get_altloc() == "A"  or atom.get_altloc() == "1" 

tmp_dir= masif_opts['tmp_dir']
out_filename1=out_filename1 = tmp_dir+"/"+pdb
infilename=pdb_filename
outfilename=out_filename1

parser = PDBParser(QUIET=True)
struct = parser.get_structure(infilename, infilename)

model = Selection.unfold_entities(struct, "M")[0]
structBuild = StructureBuilder.StructureBuilder()
structBuild.init_structure("output")
structBuild.init_seg(" ")
structBuild.init_model(0)
outputStruct = structBuild.get_structure()

for chain in model:
    structBuild.init_chain(chain.get_id())
    for residue in chain:
        het = residue.get_id()
        if het[0] == " ":
            outputStruct[0][chain.get_id()].add(residue)
pdbio = PDBIO()
pdbio.set_structure(outputStruct)
pdbio.save(outfilename, select=NotDisordered())

In [ ]:
model = Selection.unfold_entities(struct, "M")[0]
structBuild = StructureBuilder.StructureBuilder()
structBuild.init_structure("output")
structBuild.init_seg(" ")
structBuild.init_model(0)
outputStruct = structBuild.get_structure()

for chain in model:
    structBuild.init_chain(chain.get_id())
    for residue in chain:
        het = residue.get_id()
        if het[0] == " ":
            outputStruct[0][chain.get_id()].add(residue)
pdbio = PDBIO()
pdbio.set_structure(outputStruct)
pdbio.save(outfilename, select=NotDisordered())

In [ ]:
gases_ligands = ['O2I', 'AKG']
ppdb_df._df['HETATM']

In [ ]:
sel.save(masif_opts["ligand"]["assembly_dir"]+pdb)

In [ ]:
sel.save(masif_opts["ligand"]["assembly_dir"]+pdb)
ppdb_df = PandasPdb().read_pdb(masif_opts["ligand"]["assembly_dir"]+pdb)
o2if = ppdb_df.df['ATOM']
o2if[o2if.residue_name.isin(gases_ligands)]

In [ ]:
ppdb_df.df['HETATM']

In [ ]:
ppdb_df._df

In [ ]:
# sel.save(masif_opts["ligand"]["assembly_dir"]+pdb)
         


In [ ]:
ppdb_df = PandasPdb().read_pdb(masif_opts["ligand"]["assembly_dir"]+pdb)
o2if = ppdb_df.df['ATOM']
o2if.loc[o2if.residue_name=='O2I','atom_name'] = np.concatenate([['OI1','OI2']] * (len(o2if.loc[o2if.residue_name=='O2I','atom_name'])//2))
ppdb_df._df['ATOM'] = o2if[(o2if['residue_name']!='ACE') & (o2if['residue_name']!='HOH')]
ppdb_df.to_pdb(masif_opts["ligand"]["assembly_dir"]+'/'+pdb)


In [ ]:
# which frames have O2IF within 2.5 angstroms of protein or within 9 of AKG
protein = traj.topology.select('protein')
gas2 = traj.topology.select('resname %s' % gas)
AKG = traj.topology.select('resname AKG')


def cmap_per_residue(traj,sel_list,ref_list,cutoff=3.5,metric='euclidean'):
    if len(ref_list)==0 or len(sel_list)==0:
        print("One of the selections is empty")
        return []
    coord1 = traj.xyz[:,sel_list,:]*10.0 # gas
    coord2 = traj.xyz[:,ref_list,:]*10.0 # protein
    residue_sel = np.array([traj.topology.atom(ind).residue.index for ind in sel_list]) # gas
    residue_ref = np.array([traj.topology.atom(ind).residue.index for ind in ref_list])
    residue_sel_un = np.unique(residue_sel) # gas
    residue_ref_un = np.unique(residue_ref)
    dic_sel={t:c  for c,t in enumerate(residue_sel_un)} # gas
    dic_ref={t:c  for c,t in enumerate(residue_ref_un)}
    histo = np.zeros((len(coord1),len(residue_sel_un),len(residue_ref_un)))
    for c,j in enumerate(coord1):
        axis_sel,axis_ref = np.where(scipy.spatial.distance.cdist(coord1[c],coord2[c],metric=metric) < cutoff)
        real_a= residue_sel[axis_sel]
        real_b= residue_ref[axis_ref]
        elements,counts = np.unique(list(zip(real_a,real_b)),axis=0,return_counts=True)
        for c1,el in enumerate(elements):
            ax1=dic_sel[el[0]];
            ax2=dic_ref[el[1]];
            histo[c,ax1,ax2] = counts[c1];
    return residue_sel_un,residue_ref_un,histo




In [ ]:
time=np.arange(len(traj))*.01
len(time)

In [ ]:
test = cmap_per_residue(traj,gas2,protein,cutoff=2.5)
nfrcond=2

test = cmap_per_residue(traj,gas2,protein,cutoff=2.5)
segments_dict = {}
for i in range(0,24):
    res=test[2][0:,i]
    t=np.zeros(len(traj))
    t[np.where(res)[0]] = 1

    segment=[]
    subsegment,subtime,subframe=[],[],[]
    for c,value in enumerate(t):
        if value==0 and len(subsegment)==0:
            subsegment,subtime=[],[]
        elif value==0 and len(subsegment)!=0:
            if len(subsegment) > nfrcond: 
                segment.append(np.array([subframe,subtime,subsegment]))
            subsegment,subtime,subframe=[],[],[]
        else: 
            subsegment.append(value)
            subtime.append(time[c])
            subframe.append(c)
    if len(subsegment) > nfrcond: segment.append(np.array([subframe,subtime,subsegment]))
    segments_dict[i] = segment

l=[]
for k in segments_dict.keys():
    if len(segments_dict[k]) > 0:
        sl = np.concatenate([r[0] for r in segments_dict[k]])
        l.append(np.unique(sl))
frames = np.unique(np.concatenate(l).astype(int))
fn = frames+1
print(fn)
trajf=traj[frames]

for i,f in enumerate(frames):
    trajfn = trajf[i]
    test2 = cmap_per_residue(trajfn,gas2,protein,cutoff=2.5)
    whr = np.where([j.sum()>0 for j in test2[2][0]])[0]*2
    whr2 = np.concatenate([(n,(n+1)) for n in whr])
    print(whr2)
    atms = np.sort(np.concatenate((gas2[whr2],AKG,protein)))
    sel = trajfn.atom_slice(atms)
    atms = np.sort(np.concatenate((gas2[whr2],AKG,protein)))
    sel = trajfn.atom_slice(atms)
    pdb="Sim_%d_traj_%d_frame_%d" % (simnum, trjnum, f+1)+'.pdb'
    sel.save(masif_opts["ligand"]["assembly_dir"]+pdb)
    
    ppdb_df = PandasPdb().read_pdb(masif_opts["ligand"]["assembly_dir"]+pdb)
    o2if = ppdb_df.df['ATOM']
    o2if.loc[o2if.residue_name=='O2I','atom_name'] = np.concatenate([['OI1','OI2']] * (len(o2if.loc[o2if.residue_name=='O2I','atom_name'])//2))
    ppdb_df._df['ATOM'] = o2if[(o2if['residue_name']!='ACE') & (o2if['residue_name']!='HOH')]
    ppdb_df.to_pdb(masif_opts["ligand"]["assembly_dir"]+'/'+pdb)


In [ ]:
for i,f in enumerate(frames):
    trajfn = trajf[i]
    test2 = cmap_per_residue(trajfn,gas2,protein,cutoff=2.5)
    whr = np.where([j.sum()>0 for j in test2[2][0]])[0]*2
    whr2 = np.concatenate([(n,(n+1)) for n in whr])
    print(whr2)
    atms = np.sort(np.concatenate((gas2[whr2],AKG,protein)))
    sel = trajfn.atom_slice(atms)
    atms = np.sort(np.concatenate((gas2[whr2],AKG,protein)))
    sel = trajfn.atom_slice(atms)
    pdb="Sim_%d_traj_%d_frame_%d" % (simnum, trjnum, f+1)+'.pdb'
    sel.save(masif_opts["ligand"]["assembly_dir"]+pdb)
    
    ppdb_df = PandasPdb().read_pdb(masif_opts["ligand"]["assembly_dir"]+pdb)
    o2if = ppdb_df.df['ATOM']
    o2if.loc[o2if.residue_name=='O2I','atom_name'] = np.concatenate([['OI1','OI2']] * (len(o2if.loc[o2if.residue_name=='O2I','atom_name'])//2))
    ppdb_df._df['ATOM'] = o2if[(o2if['residue_name']!='ACE') & (o2if['residue_name']!='HOH')]
    ppdb_df.to_pdb(masif_opts["ligand"]["assembly_dir"]+'/'+pdb)

In [ ]:
df=ppdb_df._df['ATOM']
o2i_uniq = np.unique(df[df.residue_name=='O2I'].residue_number)
o2i_uniq = np.sort(o2i_uniq)

lig_types = []
akg_rn = np.unique(df[df.residue_name=='AKG'].residue_number)
lig_types.append(['AKG','res_%d' % akg_rn])
[lig_types.append([gas, 'res_%d' % i]) for i in o2i_uniq]
np.save(masif_opts["ligand"]["ligand_coords_dir"]+pdb.split('.')[0]+'_ligtypes.npy', lig_types)

lig_coords = {}
lig_coords[akg_rn[0]] = df[df.residue_name=='AKG'][['x_coord', 'y_coord','z_coord']].values
for val in o2i_uniq:
    print(val)
    lig_coords[val] = df[df.residue_number==val][['x_coord', 'y_coord','z_coord']].values

np.save(masif_opts["ligand"]["ligand_coords_dir"]+pdb.split('.')[0]+'_ligcoords.npy', lig_coords)



```bash
rm $prefix.pdb
files=$(ls Sim_2*_traj_2*frame**.pdb | sort -V)
prefix=Sim_2_traj_2
for f in $files; do
    echo $(wc -l $f)
    frame=$(echo $f | awk -F_ '{print $NF}' | cut -d. -f1)
    echo "MODEL $frame" >> $prefix.pdb
    cat $f | grep -v MODEL | grep -v 'END$' >> $prefix.pdb
done
echo END >> $prefix.pdb
